# 🎓 RIS PhD Ultimate Research Dashboard

## Comprehensive, Fully Customizable Research Platform

**Version:** 1.0.0  
**Purpose:** Professional-grade research platform for RIS probe-based ML experiments

### Features:
- ✅ 5 Customization Tabs (System, Model, Training, Evaluation, Visualization)
- ✅ 19 Pre-defined Model Architectures + Custom
- ✅ 6 Probe Types (continuous, binary, 2bit, hadamard, sobol, halton)
- ✅ 25+ Plot Types
- ✅ Multi-Model Comparison
- ✅ Multi-Seed Statistical Analysis
- ✅ Config Save/Load (JSON/YAML)

---

## Cell 1: Setup & Installation Check

Verify environment setup and dependencies.

In [1]:
# CELL 1: SETUP
%load_ext autoreload
%autoreload 2

import sys
import os

# 1. Ensure Project Root is in Path
project_root = os.path.abspath('.')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# 2. Import Dashboard Components
import dashboard
from dashboard.widgets import (
    get_all_widgets,
    create_unified_dashboard,
    create_results_area
)
from dashboard.callbacks import (
    setup_all_callbacks,
    setup_experiment_handlers
)

# 3. Initialize & Connect
print("🔄 Initializing RIS Research Platform...")

# Fetch all UI elements
widgets_dict = get_all_widgets()

# Connect Logic:
# a. Attach standard listeners (sliders, dropdowns updates)
setup_all_callbacks(widgets_dict)

# b. Attach Workflow Handlers (Add to Stack, Run Stack, Exports)
setup_experiment_handlers(widgets_dict)

print(f"✅ Dashboard v{dashboard.__version__} Ready!")
print("   ✓ Stacking Workflow: Active")
print("   ✓ Transfer Learning: Active")
print("   ✓ Decoupled Plotting: Active")
# Configure experiments, build your stack, and execute.
display(create_unified_dashboard())


🔄 Initializing RIS Research Platform...
✅ Dashboard v1.2.0 Ready!
   ✓ Stacking Workflow: Active
   ✓ Transfer Learning: Active
   ✓ Decoupled Plotting: Active


In [ ]:
# CELL 3: RESULTS DASHBOARD
# View analysis, change plot types, and export data.
display(create_results_area())

In [ ]:
import os
import sys

# Ensure project root is in path so we can import our modules
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from config import get_config
from experiments.probe_generators import get_probe_bank # Use the factory function
from data_generation import create_dataloaders

# 1. Setup minimal config
# Ensure N matches your MATLAB file (N=32)
cfg = get_config(system={'N': 32, 'K': 64, 'M': 8})

# 2. Create the ProbeBank properly
# Instead of ProbeBank(N, K), we use the generator which creates the 'phases'
pb = get_probe_bank(
    probe_type=cfg.system.probe_type, # e.g., 'continuous' or 'hadamard'
    N=cfg.system.N,
    K=cfg.system.K,
    seed=cfg.data.seed
)

# 3. Run the loader explicitly
# This triggers the print statements in data_generation.py
try:
    train_l, val_l, test_l, meta = create_dataloaders(cfg, pb)
    print("\n" + "="*50)
    print("✅ VERIFICATION SUCCESSFUL")
    print(f"   Dataloaders are using MATLAB ground truth.")
    print(f"   Train samples: {len(train_l.dataset)}")
    print(f"   Input shape:   {train_l.dataset.inputs.shape}")
    print("="*50)
except Exception as e:
    print("\n" + "="*50)
    print(f"❌ VERIFICATION FAILED: {str(e)}")
    print("="*50)

### Parameter Reference Table

| Category | Parameter | Default | Range | Description |
|----------|-----------|---------|-------|-------------|
| **System** | N | 32 | 4-256 | Number of RIS elements |
| | K | 64 | 4-512 | Total probes in codebook |
| | M | 8 | 1-K | Sensing budget (probes measured) |
| | P_tx | 1.0 | 0.1-10 | Transmit power |
| | probe_type | continuous | 6 options | Probe generation method |
| **Model** | hidden_sizes | [512,256,128] | varies | Layer architecture |
| | dropout_prob | 0.1 | 0-0.8 | Dropout regularization |
| **Training** | n_train | 50000 | 1000+ | Training samples |
| | learning_rate | 1e-3 | 1e-5 to 1e-1 | Learning rate |
| | batch_size | 128 | 32-512 | Batch size |
| | n_epochs | 50 | 1-500 | Maximum epochs |
| **Eval** | top_m_values | [1,2,4,8] | 1-K | Top-m accuracy metrics |

---

## 📚 Learning Guide

### Probe Types Explained

1. **Continuous**: Random phases in [0, 2π). Best for theoretical studies.
2. **Binary**: Phases {0, π}. Simplest hardware implementation.
3. **2-bit**: Phases {0, π/2, π, 3π/2}. Good balance of performance and simplicity.
4. **Hadamard**: Structured orthogonal patterns. Excellent diversity.
5. **Sobol**: Low-discrepancy quasi-random. Better coverage than random.
6. **Halton**: Another quasi-random sequence. Similar to Sobol.

### Model Architecture Guidelines

- **Wider networks** (e.g., DoubleWide): More capacity, risk of overfitting
- **Deeper networks** (e.g., VeryDeep): Better feature extraction, harder to train
- **Pyramidal** (e.g., Pyramid): Natural information compression
- **Hourglass**: Information bottleneck for robust features
- **ResNet-style**: Same width, easier gradient flow

### Key Parameter Interactions

- **M/K ratio**: Critical for performance. Lower ratio = harder problem.
- **Learning rate**: Most important hyperparameter. Start with 1e-3.
- **Dropout**: Use 0.1-0.2 for regularization. Higher values for larger models.
- **Batch size**: Larger = more stable gradients. Smaller = better generalization.

### Typical Workflows

1. **Quick Test**: Default settings, 1 epoch, check if system works
2. **Architecture Search**: Compare multiple models, same data/training
3. **Hyperparameter Tuning**: Fix architecture, sweep learning rates
4. **Statistical Validation**: Multi-seed runs for confidence intervals
5. **Publication Results**: Best config, full training, all plots

---

## 🛠️ Troubleshooting

**Out of memory?**
- Reduce batch_size
- Use smaller model
- Reduce n_train

**Training too slow?**
- Reduce n_epochs
- Use smaller dataset
- Use simpler model

**Poor performance?**
- Increase model capacity
- Try different probe types
- Adjust learning rate
- More training data

**Overfitting?**
- Increase dropout
- Add weight_decay
- Reduce model size
- More training data

---

## 📖 References

For more information, see:
- `dashboard/README.md` - Detailed documentation
- `EXTENSION_GUIDE.md` - How to extend the system
- `USAGE_EXAMPLES.md` - Usage examples

---

**Happy Researching! 🚀**

In [2]:
import os
os.getcwd()
%cd ..

C:\Users\els25maa\Esperiments\Session6_Probe_Different_Approaches\Jupyter\session5_recovery\recovery_Opus\Session5_clone


In [3]:
# ============================================================================
# STAGE 1: FOLDER STRUCTURE SETUP SCRIPT
# Run this in a Jupyter cell to create the clean architecture
# ============================================================================

import os
from pathlib import Path

def create_clean_architecture():
    """Create professional folder structure for PhD RIS project."""

    print("="*70)
    print("CREATING CLEAN PROJECT ARCHITECTURE")
    print("="*70)
    print()

    # Define folder structure
    folders = [
        "config",
        "physics",  # Already exists
        "models",
        "data",
        "training",
        "evaluation",
        "dashboard",
        "dashboard/tabs",
        "dashboard/components",
        "plotting",
        "notebooks/experiments",
        "results",
        "tests",
        "docs"
    ]

    # Create folders
    for folder in folders:
        Path(folder).mkdir(parents=True, exist_ok=True)
        print(f"[OK] Created: {folder}/")

        # Create __init__.py for Python packages
        if folder not in ["results", "notebooks", "notebooks/experiments", "docs", "tests"]:
            init_file = Path(folder) / "__init__.py"
            if not init_file.exists():
                init_file.write_text(f'"""\\n{folder.replace("/", ".")} module\\n"""\\n')
                print(f"     + {folder}/__init__.py")

    print()
    print("="*70)
    print("FOLDER STRUCTURE CREATED SUCCESSFULLY")
    print("="*70)
    print()
    print("Next: Run Stage 2 to create tab files...")

# Run it
create_clean_architecture()

CREATING CLEAN PROJECT ARCHITECTURE

[OK] Created: config/
     + config/__init__.py
[OK] Created: physics/
[OK] Created: models/
     + models/__init__.py
[OK] Created: data/
     + data/__init__.py
[OK] Created: training/
     + training/__init__.py
[OK] Created: evaluation/
     + evaluation/__init__.py
[OK] Created: dashboard/
[OK] Created: dashboard/tabs/
     + dashboard/tabs/__init__.py
[OK] Created: dashboard/components/
     + dashboard/components/__init__.py
[OK] Created: plotting/
     + plotting/__init__.py
[OK] Created: notebooks/experiments/
[OK] Created: results/
[OK] Created: tests/
[OK] Created: docs/

FOLDER STRUCTURE CREATED SUCCESSFULLY

Next: Run Stage 2 to create tab files...


In [4]:
# ============================================================================
# STAGE 2: CREATE TAB FILES
# ============================================================================

from pathlib import Path

def create_tab_files():
    """Create individual tab files in dashboard/tabs/."""

    print("Creating tab files...\n")

    # ========================================================================
    # TAB 1: SYSTEM
    # ========================================================================

    tab_system_content = '''"""
Tab 1: Core System Parameters
==============================
N, K, M, transmit power, channel variances, phase configuration, probe types.
"""

import ipywidgets as widgets


def create_system_tab():
    """Create Core System tab with essential parameters."""

    # RIS Configuration
    widget_N = widgets.IntSlider(
        value=32, min=4, max=256, step=1,
        description='N (RIS elements):',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px'),
        tooltip='Number of reconfigurable elements on the RIS'
    )

    widget_K = widgets.IntSlider(
        value=64, min=4, max=512, step=1,
        description='K (Codebook size):',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px'),
        tooltip='Total number of probe configurations in the bank'
    )

    widget_M = widgets.IntSlider(
        value=8, min=1, max=64, step=1,
        description='M (Sensing budget):',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px'),
        tooltip='Number of probes measured per channel realization'
    )

    # M/K ratio indicator
    ratio_display = widgets.HTML(
        value="<div style='margin-left: 190px; color: #666;'><i>M/K ratio: 12.5% (sparse sensing)</i></div>",
        layout=widgets.Layout(width='500px')
    )

    def update_ratio(change=None):
        M_val = widget_M.value
        K_val = widget_K.value
        ratio = M_val / K_val * 100

        if ratio < 10:
            color, label = "#d32f2f", "very sparse"
        elif ratio < 25:
            color, label = "#f57c00", "sparse"
        elif ratio < 50:
            color, label = "#fbc02d", "moderate"
        else:
            color, label = "#388e3c", "dense"

        ratio_display.value = (
            f"<div style='margin-left: 190px; color: {color};'>"
            f"<b>M/K ratio: {ratio:.1f}%</b> ({label} sensing)"
            f"</div>"
        )

    widget_M.observe(update_ratio, 'value')
    widget_K.observe(update_ratio, 'value')

    # Channel Physics
    widget_P_tx = widgets.FloatSlider(
        value=1.0, min=0.1, max=10.0, step=0.1,
        description='P_tx (Transmit power):',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    widget_sigma_h_sq = widgets.FloatSlider(
        value=1.0, min=0.1, max=10.0, step=0.1,
        description='σ_h² (BS-RIS variance):',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    widget_sigma_g_sq = widgets.FloatSlider(
        value=1.0, min=0.1, max=10.0, step=0.1,
        description='σ_g² (RIS-UE variance):',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    # Phase Configuration
    widget_phase_mode = widgets.Dropdown(
        options=['continuous', 'discrete'],
        value='continuous',
        description='Phase mode:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    widget_phase_bits = widgets.IntSlider(
        value=3, min=1, max=8, step=1,
        description='Phase bits:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px'),
        disabled=True
    )

    def toggle_phase_bits(change):
        widget_phase_bits.disabled = (change['new'] == 'continuous')

    widget_phase_mode.observe(toggle_phase_bits, 'value')

    # Probe Configuration
    widget_probe_category = widgets.Dropdown(
        options=['Physics-Based', 'Mathematical Sequence'],
        value='Physics-Based',
        description='Probe Category:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    widget_probe_type = widgets.Dropdown(
        options=['continuous', 'binary', '2bit'],
        value='continuous',
        description='Probe Type:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    def update_probe_options(change):
        if change['new'] == 'Physics-Based':
            widget_probe_type.options = ['continuous', 'binary', '2bit']
        else:
            widget_probe_type.options = ['hadamard', 'sobol', 'halton']
        widget_probe_type.value = widget_probe_type.options[0]

    widget_probe_category.observe(update_probe_options, 'value')

    # Layout
    tab_layout = widgets.VBox([
        widgets.HTML("<h3>Core System Parameters</h3>"),
        widgets.HTML("<h4>RIS Configuration</h4>"),
        widget_N,
        widget_K,
        widget_M,
        ratio_display,
        widgets.HTML("<h4 style='margin-top: 20px;'>Channel Physics</h4>"),
        widget_P_tx,
        widget_sigma_h_sq,
        widget_sigma_g_sq,
        widgets.HTML("<h4 style='margin-top: 20px;'>Phase Configuration</h4>"),
        widget_phase_mode,
        widget_phase_bits,
        widgets.HTML("<h4 style='margin-top: 20px;'>Probe Configuration</h4>"),
        widget_probe_category,
        widget_probe_type
    ], layout=widgets.Layout(padding='20px'))

    # Store widget references for external access
    tab_layout._widgets = {
        'N': widget_N,
        'K': widget_K,
        'M': widget_M,
        'P_tx': widget_P_tx,
        'sigma_h_sq': widget_sigma_h_sq,
        'sigma_g_sq': widget_sigma_g_sq,
        'phase_mode': widget_phase_mode,
        'phase_bits': widget_phase_bits,
        'probe_category': widget_probe_category,
        'probe_type': widget_probe_type
    }

    return tab_layout
'''

    with open("dashboard/tabs/tab_system.py", "w", encoding='utf-8') as f:
        f.write(tab_system_content)
    print("[OK] Created: dashboard/tabs/tab_system.py")

    # ========================================================================
    # TAB 2: PHYSICS & REALISM
    # ========================================================================

    tab_physics_content = '''"""
Tab 2: Physics & Realism Configuration
=======================================
Channel sources, realism profiles, advanced impairment settings.
"""

import ipywidgets as widgets


def create_physics_tab():
    """Create Physics & Realism tab."""

    # Channel Source
    widget_channel_source = widgets.Dropdown(
        options=[
            ('Python Synthetic (Built-in)', 'python_synthetic'),
            ('MATLAB Engine (Phase 2)', 'matlab_engine'),
            ('MATLAB Verified Data (Phase 2)', 'matlab_verified')
        ],
        value='python_synthetic',
        description='Channel Source:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='600px'),
        tooltip='Select physics simulation backend'
    )

    # Source info display
    widget_source_info = widgets.HTML(
        value=(
            "<div style='margin-left: 190px; padding: 10px; background: #e3f2fd; border-left: 3px solid #2196f3;'>"
            "<b>Python Synthetic</b><br>"
            "Built-in numpy-based Rayleigh fading<br>"
            "[OK] Analytically verified<br>"
            "[OK] Fast and reliable"
            "</div>"
        ),
        layout=widgets.Layout(width='600px')
    )

    def update_source_info(change):
        source = change['new']

        if source == 'python_synthetic':
            info = (
                "<div style='margin-left: 190px; padding: 10px; background: #e3f2fd; border-left: 3px solid #2196f3;'>"
                "<b>Python Synthetic</b><br>"
                "Built-in numpy-based Rayleigh fading<br>"
                "[OK] Analytically verified<br>"
                "[OK] Fast and reliable"
                "</div>"
            )
        elif source == 'matlab_engine':
            info = (
                "<div style='margin-left: 190px; padding: 10px; background: #fff3e0; border-left: 3px solid #ff9800;'>"
                "<b>MATLAB Engine (Not Implemented)</b><br>"
                "Live MATLAB channel generation<br>"
                "[WARN] Requires MATLAB R2021b+<br>"
                "[WARN] Coming in Phase 2"
                "</div>"
            )
        else:
            info = (
                "<div style='margin-left: 190px; padding: 10px; background: #fff3e0; border-left: 3px solid #ff9800;'>"
                "<b>MATLAB Verified Data (Not Implemented)</b><br>"
                "Load pre-verified .mat files<br>"
                "[WARN] Requires pre-generated data<br>"
                "[WARN] Coming in Phase 2"
                "</div>"
            )

        widget_source_info.value = info

    widget_channel_source.observe(update_source_info, 'value')

    # Realism Profile
    widget_realism_profile = widgets.Dropdown(
        options=[
            ('Ideal (No Impairments)', 'ideal'),
            ('Mild Impairments (Lab)', 'mild_impairments'),
            ('Moderate Impairments (Indoor)', 'moderate_impairments'),
            ('Severe Impairments (Outdoor)', 'severe_impairments'),
            ('Worst Case (Stress Test)', 'worst_case')
        ],
        value='ideal',
        description='Realism Profile:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='600px'),
        tooltip='Select pre-configured impairment bundle'
    )

    # Profile descriptions
    profile_descriptions = {
        'ideal': (
            "<b>Ideal Conditions</b><br>"
            "• Perfect CSI<br>"
            "• Infinite precision hardware<br>"
            "• No environmental effects<br>"
            "<i>Use for: Theoretical upper bounds</i>"
        ),
        'mild_impairments': (
            "<b>Mild Impairments</b><br>"
            "• -30 dB CSI error (0.1%)<br>"
            "• 5 Hz Doppler, 10ms delay<br>"
            "• 6-bit phase shifters<br>"
            "<i>Use for: High-quality lab equipment</i>"
        ),
        'moderate_impairments': (
            "<b>Moderate Impairments</b><br>"
            "• -20 dB CSI error (1%)<br>"
            "• 10 Hz Doppler, 20ms delay<br>"
            "• 4-bit phase shifters<br>"
            "<i>Use for: Typical indoor deployment</i>"
        ),
        'severe_impairments': (
            "<b>Severe Impairments</b><br>"
            "• -15 dB CSI error (3%)<br>"
            "• 50 Hz Doppler, 50ms delay<br>"
            "• 3-bit phase shifters<br>"
            "<i>Use for: Outdoor/vehicular scenarios</i>"
        ),
        'worst_case': (
            "<b>Worst Case</b><br>"
            "• -10 dB CSI error (10%)<br>"
            "• 100 Hz Doppler, 100ms delay<br>"
            "• 2-bit phase shifters<br>"
            "<i>Use for: Robustness testing</i>"
        )
    }

    widget_profile_info = widgets.HTML(
        value=(
            "<div style='margin-left: 190px; padding: 10px; background: #e8f5e9; border-left: 3px solid #4caf50;'>"
            + profile_descriptions['ideal'] +
            "</div>"
        ),
        layout=widgets.Layout(width='600px')
    )

    def update_profile_info(change):
        profile = change['new']

        color_map = {
            'ideal': ('#e8f5e9', '#4caf50'),
            'mild_impairments': ('#fff9c4', '#fbc02d'),
            'moderate_impairments': ('#ffe0b2', '#f57c00'),
            'severe_impairments': ('#ffccbc', '#d84315'),
            'worst_case': ('#f3e5f5', '#7b1fa2')
        }

        bg_color, border_color = color_map.get(profile, ('#fff', '#999'))

        widget_profile_info.value = (
            f"<div style='margin-left: 190px; padding: 10px; background: {bg_color}; border-left: 3px solid {border_color};'>"
            + profile_descriptions.get(profile, "Unknown profile") +
            "</div>"
        )

    widget_realism_profile.observe(update_profile_info, 'value')

    # Advanced Impairments
    widget_use_custom_impairments = widgets.Checkbox(
        value=False,
        description='Advanced: Use Custom Impairments',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='600px', margin='20px 0 10px 0')
    )

    widget_csi_error_db = widgets.FloatSlider(
        value=-20, min=-40, max=-5, step=1,
        description='CSI Error (dB):',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px'),
        disabled=True
    )

    widget_doppler_hz = widgets.FloatSlider(
        value=10, min=0, max=200, step=5,
        description='Doppler (Hz):',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px'),
        disabled=True
    )

    widget_phase_bits_hw = widgets.IntSlider(
        value=4, min=1, max=8, step=1,
        description='Phase Shifter Bits:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px'),
        disabled=True
    )

    widget_adc_bits = widgets.IntSlider(
        value=10, min=6, max=16, step=1,
        description='ADC Bits:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px'),
        disabled=True
    )

    advanced_warning = widgets.HTML(
        value=(
            "<div style='margin: 10px 0 10px 190px; padding: 10px; background: #ffebee; border-left: 3px solid #f44336;'>"
            "[WARNING] Modifying these settings will override the selected realism profile. "
            "Custom configurations are logged separately."
            "</div>"
        ),
        layout=widgets.Layout(width='600px')
    )

    advanced_container = widgets.VBox([
        advanced_warning,
        widget_csi_error_db,
        widget_doppler_hz,
        widget_phase_bits_hw,
        widget_adc_bits
    ])
    advanced_container.layout.display = 'none'

    def toggle_advanced(change):
        is_custom = change['new']
        advanced_container.layout.display = 'block' if is_custom else 'none'
        widget_csi_error_db.disabled = not is_custom
        widget_doppler_hz.disabled = not is_custom
        widget_phase_bits_hw.disabled = not is_custom
        widget_adc_bits.disabled = not is_custom

    widget_use_custom_impairments.observe(toggle_advanced, 'value')

    # Layout
    tab_layout = widgets.VBox([
        widgets.HTML("<h3>Physics & Realism Configuration</h3>"),
        widgets.HTML("<h4>Channel Source</h4>"),
        widget_channel_source,
        widget_source_info,
        widgets.HTML("<h4 style='margin-top: 20px;'>Realism Profile</h4>"),
        widget_realism_profile,
        widget_profile_info,
        widgets.HTML("<hr style='margin: 20px 0;'>"),
        widget_use_custom_impairments,
        advanced_container
    ], layout=widgets.Layout(padding='20px'))

    # Store widget references
    tab_layout._widgets = {
        'channel_source': widget_channel_source,
        'source_info': widget_source_info,
        'realism_profile': widget_realism_profile,
        'profile_info': widget_profile_info,
        'use_custom_impairments': widget_use_custom_impairments,
        'csi_error_db': widget_csi_error_db,
        'doppler_hz': widget_doppler_hz,
        'phase_bits_hw': widget_phase_bits_hw,
        'adc_bits': widget_adc_bits
    }

    return tab_layout
'''

    with open("dashboard/tabs/tab_physics.py", "w", encoding='utf-8') as f:
        f.write(tab_physics_content)
    print("[OK] Created: dashboard/tabs/tab_physics.py")

    # ========================================================================
    # TAB 3-6: Placeholder files (I'll create full versions in next stage)
    # ========================================================================

    placeholder_tabs = {
        'tab_model.py': 'Model Architecture',
        'tab_training.py': 'Training Configuration',
        'tab_evaluation.py': 'Evaluation & Comparison',
        'tab_visualization.py': 'Visualization Control'
    }

    for filename, tab_name in placeholder_tabs.items():
        content = f'''"""
{tab_name} Tab
{'='*len(tab_name)}
TODO: Migrate from original widgets.py
"""

import ipywidgets as widgets


def create_{filename.replace("tab_", "").replace(".py", "")}_tab():
    """Create {tab_name} tab."""

    # Placeholder - will be populated in next stage
    tab_layout = widgets.VBox([
        widgets.HTML(f"<h3>{tab_name}</h3>"),
        widgets.HTML("<p><i>Migrating from original widgets.py...</i></p>")
    ], layout=widgets.Layout(padding='20px'))

    tab_layout._widgets = {{}}

    return tab_layout
'''

        with open(f"dashboard/tabs/{filename}", "w", encoding='utf-8') as f:
            f.write(content)
        print(f"[OK] Created: dashboard/tabs/{filename} (placeholder)")

    # ========================================================================
    # Create tabs/__init__.py
    # ========================================================================

    tabs_init_content = '''"""
Dashboard Tabs Module
=====================
Each tab in a separate, maintainable file.
"""

from dashboard.tabs.tab_system import create_system_tab
from dashboard.tabs.tab_physics import create_physics_tab
from dashboard.tabs.tab_model import create_model_tab
from dashboard.tabs.tab_training import create_training_tab
from dashboard.tabs.tab_evaluation import create_evaluation_tab
from dashboard.tabs.tab_visualization import create_visualization_tab

__all__ = [
    'create_system_tab',
    'create_physics_tab',
    'create_model_tab',
    'create_training_tab',
    'create_evaluation_tab',
    'create_visualization_tab'
]
'''

    with open("dashboard/tabs/__init__.py", "w", encoding='utf-8') as f:
        f.write(tabs_init_content)
    print("[OK] Created: dashboard/tabs/__init__.py")

    print()
    print("="*70)
    print("TAB FILES CREATED SUCCESSFULLY")
    print("="*70)
    print()
    print("Next: Run Stage 3 to create component files...")

# Run it
create_tab_files()

Creating tab files...

[OK] Created: dashboard/tabs/tab_system.py
[OK] Created: dashboard/tabs/tab_physics.py
[OK] Created: dashboard/tabs/tab_model.py (placeholder)
[OK] Created: dashboard/tabs/tab_training.py (placeholder)
[OK] Created: dashboard/tabs/tab_evaluation.py (placeholder)
[OK] Created: dashboard/tabs/tab_visualization.py (placeholder)
[OK] Created: dashboard/tabs/__init__.py

TAB FILES CREATED SUCCESSFULLY

Next: Run Stage 3 to create component files...


In [5]:
# ============================================================================
# STAGE 3: CREATE COMPONENT FILES
# ============================================================================

from pathlib import Path

def create_component_files():
    """Create reusable UI component files."""

    print("Creating component files...\n")

    # ========================================================================
    # COMPONENT 1: Stack Manager
    # ========================================================================

    stack_manager_content = '''"""
Stack Manager Component
=======================
Experiment stack UI and management logic.
"""

import ipywidgets as widgets


def create_stack_manager():
    """Create experiment stack manager component."""

    # Custom experiment name input
    widget_custom_exp_name = widgets.Text(
        value='',
        placeholder='Enter custom experiment name (optional)',
        description='Custom Name:',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='100%'),
        tooltip='Leave empty for auto-generated name'
    )

    # Stack display (list of experiments)
    widget_stack_display = widgets.Select(
        options=[],
        description='Experiment Stack:',
        layout=widgets.Layout(width='100%', height='150px'),
        tooltip='Select experiments to remove or reorder'
    )

    # Stack control buttons
    button_add_to_stack = widgets.Button(
        description='ADD TO STACK',
        button_style='primary',
        layout=widgets.Layout(width='140px'),
        icon='plus',
        tooltip='Add current configuration to stack'
    )

    button_remove_from_stack = widgets.Button(
        description='REMOVE',
        button_style='danger',
        layout=widgets.Layout(width='120px'),
        icon='minus',
        tooltip='Remove selected experiment from stack'
    )

    button_move_up = widgets.Button(
        description='UP',
        button_style='info',
        layout=widgets.Layout(width='100px'),
        icon='arrow-up',
        tooltip='Move selected experiment up'
    )

    button_move_down = widgets.Button(
        description='DOWN',
        button_style='info',
        layout=widgets.Layout(width='100px'),
        icon='arrow-down',
        tooltip='Move selected experiment down'
    )

    button_clear_stack = widgets.Button(
        description='CLEAR ALL',
        button_style='warning',
        layout=widgets.Layout(width='120px'),
        icon='trash',
        tooltip='Clear entire stack'
    )

    button_save_stack = widgets.Button(
        description='SAVE STACK',
        button_style='success',
        layout=widgets.Layout(width='130px'),
        icon='save',
        tooltip='Save stack configuration to file'
    )

    button_load_stack = widgets.Button(
        description='LOAD STACK',
        button_style='warning',
        layout=widgets.Layout(width='130px'),
        icon='folder-open',
        tooltip='Load stack configuration from file'
    )

    button_run_stack = widgets.Button(
        description='RUN STACK',
        button_style='success',
        layout=widgets.Layout(width='100%', height='60px'),
        icon='rocket',
        tooltip='Run all experiments in stack sequentially'
    )

    button_pause_training = widgets.Button(
        description='PAUSE',
        button_style='warning',
        layout=widgets.Layout(width='100px'),
        icon='pause',
        tooltip='Pause stack execution'
    )

    button_resume_training = widgets.Button(
        description='RESUME',
        button_style='success',
        layout=widgets.Layout(width='100px'),
        icon='play',
        tooltip='Resume stack execution'
    )

    # Layout
    stack_layout = widgets.VBox([
        widgets.HTML("<b>Experiment Stack:</b> Configure parameters above, name (optional), then add to stack."),
        widget_custom_exp_name,
        widgets.HBox([
            button_add_to_stack,
            button_remove_from_stack,
            button_move_up,
            button_move_down
        ], layout=widgets.Layout(justify_content='flex-start', margin='5px 0')),
        widget_stack_display,
        widgets.HBox([
            button_save_stack,
            button_load_stack,
            button_clear_stack
        ], layout=widgets.Layout(justify_content='flex-start', margin='5px 0')),
        widgets.HBox([
            button_run_stack
        ], layout=widgets.Layout(margin='5px 0')),
        widgets.HBox([
            button_pause_training,
            button_resume_training
        ], layout=widgets.Layout(justify_content='flex-start', margin='5px 0'))
    ], layout=widgets.Layout(
        border='2px solid #673AB7',
        padding='15px',
        margin='10px 0',
        background_color='#f3e5f5'
    ))

    # Store widget references
    stack_layout._widgets = {
        'custom_exp_name': widget_custom_exp_name,
        'stack_display': widget_stack_display,
        'button_add_to_stack': button_add_to_stack,
        'button_remove_from_stack': button_remove_from_stack,
        'button_move_up': button_move_up,
        'button_move_down': button_move_down,
        'button_clear_stack': button_clear_stack,
        'button_save_stack': button_save_stack,
        'button_load_stack': button_load_stack,
        'button_run_stack': button_run_stack,
        'button_pause_training': button_pause_training,
        'button_resume_training': button_resume_training
    }

    return stack_layout
'''

    with open("dashboard/components/stack_manager.py", "w", encoding='utf-8') as f:
        f.write(stack_manager_content)
    print("[OK] Created: dashboard/components/stack_manager.py")

    # ========================================================================
    # COMPONENT 2: Action Buttons
    # ========================================================================

    buttons_content = '''"""
Action Buttons Component
========================
Main action buttons for running experiments, saving/loading configs, etc.
"""

import ipywidgets as widgets


def create_action_buttons():
    """Create main action buttons."""

    button_run_experiment = widgets.Button(
        description='RUN SINGLE',
        button_style='success',
        layout=widgets.Layout(width='150px'),
        icon='play',
        tooltip='Run single experiment with current configuration'
    )

    button_save_config = widgets.Button(
        description='SAVE CONFIG',
        button_style='info',
        layout=widgets.Layout(width='150px'),
        icon='save',
        tooltip='Save current configuration to file'
    )

    button_load_config = widgets.Button(
        description='LOAD CONFIG',
        button_style='warning',
        layout=widgets.Layout(width='150px'),
        icon='folder-open',
        tooltip='Load configuration from file'
    )

    button_reset_defaults = widgets.Button(
        description='RESET',
        button_style='danger',
        layout=widgets.Layout(width='150px'),
        icon='refresh',
        tooltip='Reset all parameters to defaults'
    )

    # Layout
    buttons_layout = widgets.HBox([
        button_run_experiment,
        button_save_config,
        button_load_config,
        button_reset_defaults
    ], layout=widgets.Layout(
        justify_content='space-around',
        padding='10px'
    ))

    # Store references
    buttons_layout._widgets = {
        'button_run_experiment': button_run_experiment,
        'button_save_config': button_save_config,
        'button_load_config': button_load_config,
        'button_reset_defaults': button_reset_defaults
    }

    return buttons_layout


def create_results_buttons():
    """Create results management buttons."""

    button_plot_only = widgets.Button(
        description='PLOT ONLY',
        button_style='info',
        layout=widgets.Layout(width='150px'),
        icon='chart-bar',
        tooltip='Generate plots without training'
    )

    button_load_results = widgets.Button(
        description='LOAD RESULTS',
        button_style='warning',
        layout=widgets.Layout(width='150px'),
        icon='folder-open',
        tooltip='Load previously saved results'
    )

    button_save_results = widgets.Button(
        description='SAVE RESULTS',
        button_style='success',
        layout=widgets.Layout(width='150px'),
        icon='save',
        tooltip='Save current results to file'
    )

    # Layout
    buttons_layout = widgets.HBox([
        button_plot_only,
        button_load_results,
        button_save_results
    ], layout=widgets.Layout(
        justify_content='center',
        padding='10px'
    ))

    # Store references
    buttons_layout._widgets = {
        'button_plot_only': button_plot_only,
        'button_load_results': button_load_results,
        'button_save_results': button_save_results
    }

    return buttons_layout


def create_export_buttons():
    """Create export buttons for results."""

    button_export_csv = widgets.Button(
        description='Export CSV',
        button_style='primary',
        layout=widgets.Layout(width='150px'),
        icon='table',
        tooltip='Export results as CSV'
    )

    button_export_json = widgets.Button(
        description='Export JSON',
        button_style='primary',
        layout=widgets.Layout(width='150px'),
        icon='code',
        tooltip='Export results as JSON'
    )

    button_export_latex = widgets.Button(
        description='Export LaTeX',
        button_style='primary',
        layout=widgets.Layout(width='150px'),
        icon='file-text',
        tooltip='Export results as LaTeX table'
    )

    button_save_model = widgets.Button(
        description='Save Model (.pt)',
        button_style='warning',
        layout=widgets.Layout(width='150px'),
        icon='save',
        tooltip='Save trained model weights'
    )

    # Layout
    buttons_layout = widgets.HBox([
        button_export_csv,
        button_export_json,
        button_export_latex,
        button_save_model
    ], layout=widgets.Layout(
        justify_content='center',
        padding='10px'
    ))

    # Store references
    buttons_layout._widgets = {
        'button_export_csv': button_export_csv,
        'button_export_json': button_export_json,
        'button_export_latex': button_export_latex,
        'button_save_model': button_save_model
    }

    return buttons_layout
'''

    with open("dashboard/components/buttons.py", "w", encoding='utf-8') as f:
        f.write(buttons_content)
    print("[OK] Created: dashboard/components/buttons.py")

    # ========================================================================
    # COMPONENT 3: Status Display
    # ========================================================================

    status_display_content = '''"""
Status Display Component
========================
Progress bars, live metrics, and log output.
"""

import ipywidgets as widgets


def create_status_display():
    """Create status display component with progress and logs."""

    # Progress bar
    widget_progress_bar = widgets.IntProgress(
        value=0,
        min=0,
        max=100,
        description='Progress:',
        bar_style='info',
        layout=widgets.Layout(width='100%')
    )

    # Live metrics display
    widget_live_metrics = widgets.HTML(
        value="<div style='font-family: monospace; padding: 10px;'><b>Waiting to start...</b></div>",
        layout=widgets.Layout(
            width='100%',
            height='150px',
            border='1px solid #ccc',
            padding='10px',
            background_color='#fafafa'
        )
    )

    # Status output (log)
    widget_status_output = widgets.Output(
        layout=widgets.Layout(
            width='100%',
            height='200px',
            border='1px solid #ccc',
            padding='10px',
            overflow='auto',
            background_color='#ffffff'
        )
    )

    # Layout
    status_layout = widgets.VBox([
        widgets.Label("Training Progress:"),
        widget_progress_bar,
        widgets.Label("Live Metrics:"),
        widget_live_metrics,
        widgets.HTML("<hr style='margin: 10px 0;'>"),
        widgets.Label("Execution Log:"),
        widget_status_output
    ], layout=widgets.Layout(
        border='1px solid #ddd',
        padding='15px',
        background_color='#f9f9f9',
        margin='10px 0'
    ))

    # Store references
    status_layout._widgets = {
        'progress_bar': widget_progress_bar,
        'live_metrics': widget_live_metrics,
        'status_output': widget_status_output
    }

    return status_layout
'''

    with open("dashboard/components/status_display.py", "w", encoding='utf-8') as f:
        f.write(status_display_content)
    print("[OK] Created: dashboard/components/status_display.py")

    # ========================================================================
    # COMPONENT 4: Results Display
    # ========================================================================

    results_display_content = '''"""
Results Display Component
=========================
Results summary, plots, and export options.
"""

import ipywidgets as widgets
from dashboard.components.buttons import create_export_buttons


def create_results_display():
    """Create results display area with tabs."""

    # Results summary HTML
    widget_results_summary = widgets.HTML(
        value="<div style='padding: 10px;'><i>No results yet. Run an experiment to see results.</i></div>",
        layout=widgets.Layout(width='100%', min_height='200px')
    )

    # Training plots output
    widget_results_plots_training = widgets.Output(
        layout=widgets.Layout(width='100%', padding='10px')
    )

    # Analysis plots output
    widget_results_plots_analysis = widgets.Output(
        layout=widgets.Layout(width='100%', padding='10px')
    )

    # Export buttons
    export_buttons = create_export_buttons()

    # Create tabs
    tab_summary = widgets.VBox([
        widget_results_summary,
        export_buttons
    ])

    tab_training = widgets.VBox([
        widget_results_plots_training
    ])

    tab_analysis = widgets.VBox([
        widget_results_plots_analysis
    ])

    results_tabs = widgets.Tab(children=[tab_summary, tab_training, tab_analysis])
    results_tabs.set_title(0, 'Summary & Export')
    results_tabs.set_title(1, 'Training Curves')
    results_tabs.set_title(2, 'Deep Analysis')

    # Main layout
    results_layout = widgets.VBox([
        widgets.HTML("<h2 style='text-align: center;'>Results & Analysis Dashboard</h2>"),
        results_tabs
    ], layout=widgets.Layout(
        padding='20px',
        border='2px solid #eee'
    ))

    # Store references
    results_layout._widgets = {
        'results_summary': widget_results_summary,
        'results_plots_training': widget_results_plots_training,
        'results_plots_analysis': widget_results_plots_analysis,
        'export_buttons': export_buttons._widgets
    }

    return results_layout
'''

    with open("dashboard/components/results_display.py", "w", encoding='utf-8') as f:
        f.write(results_display_content)
    print("[OK] Created: dashboard/components/results_display.py")

    # ========================================================================
    # Create components/__init__.py
    # ========================================================================

    components_init_content = '''"""
Dashboard Components Module
===========================
Reusable UI components for the dashboard.
"""

from dashboard.components.stack_manager import create_stack_manager
from dashboard.components.buttons import (
    create_action_buttons,
    create_results_buttons,
    create_export_buttons
)
from dashboard.components.status_display import create_status_display
from dashboard.components.results_display import create_results_display

__all__ = [
    'create_stack_manager',
    'create_action_buttons',
    'create_results_buttons',
    'create_export_buttons',
    'create_status_display',
    'create_results_display'
]
'''

    with open("dashboard/components/__init__.py", "w", encoding='utf-8') as f:
        f.write(components_init_content)
    print("[OK] Created: dashboard/components/__init__.py")

    print()
    print("="*70)
    print("COMPONENT FILES CREATED SUCCESSFULLY")
    print("="*70)
    print()
    print("Next: Run Stage 4 to create main dashboard orchestrator...")

# Run it
create_component_files()

Creating component files...

[OK] Created: dashboard/components/stack_manager.py
[OK] Created: dashboard/components/buttons.py
[OK] Created: dashboard/components/status_display.py
[OK] Created: dashboard/components/results_display.py
[OK] Created: dashboard/components/__init__.py

COMPONENT FILES CREATED SUCCESSFULLY

Next: Run Stage 4 to create main dashboard orchestrator...


In [6]:
# ============================================================================
# STAGE 4: CREATE MAIN DASHBOARD ORCHESTRATOR
# ============================================================================

from pathlib import Path

def create_dashboard_orchestrator():
    """Create main dashboard file that brings everything together."""

    print("Creating dashboard orchestrator...\n")

    # ========================================================================
    # Main Dashboard File
    # ========================================================================

    dashboard_main_content = '''"""
Main Dashboard Orchestrator
============================
Brings together all tabs, components, and creates the unified interface.
"""

import ipywidgets as widgets
from dashboard.tabs import (
    create_system_tab,
    create_physics_tab,
    create_model_tab,
    create_training_tab,
    create_evaluation_tab,
    create_visualization_tab
)
from dashboard.components import (
    create_stack_manager,
    create_action_buttons,
    create_status_display,
    create_results_display
)


def create_unified_dashboard():
    """
    Create the complete unified dashboard.

    Returns:
        tuple: (dashboard_widget, widget_dict)
            - dashboard_widget: The complete UI
            - widget_dict: Dictionary of all widgets for callbacks
    """

    # ========================================================================
    # Create Tabs
    # ========================================================================

    tab_system = create_system_tab()
    tab_physics = create_physics_tab()
    tab_model = create_model_tab()
    tab_training = create_training_tab()
    tab_evaluation = create_evaluation_tab()
    tab_visualization = create_visualization_tab()

    # Create tab container
    tabs = widgets.Tab(children=[
        tab_system,
        tab_physics,
        tab_model,
        tab_training,
        tab_evaluation,
        tab_visualization
    ])

    tabs.set_title(0, 'System')
    tabs.set_title(1, 'Physics & Realism')
    tabs.set_title(2, 'Model')
    tabs.set_title(3, 'Training')
    tabs.set_title(4, 'Evaluation')
    tabs.set_title(5, 'Visualization')

    # ========================================================================
    # Create Components (Below Tabs)
    # ========================================================================

    stack_manager = create_stack_manager()
    action_buttons = create_action_buttons()
    status_display = create_status_display()

    # ========================================================================
    # Assemble Dashboard
    # ========================================================================

    dashboard = widgets.VBox([
        tabs,                    # Parameter tabs at top
        stack_manager,           # Stack manager below tabs
        action_buttons,          # Main action buttons
        status_display           # Progress and logs
    ])

    # ========================================================================
    # Collect All Widgets into Dictionary
    # ========================================================================

    widget_dict = {}

    # Add tab widgets
    widget_dict.update(tab_system._widgets)
    widget_dict.update(tab_physics._widgets)
    widget_dict.update(tab_model._widgets)
    widget_dict.update(tab_training._widgets)
    widget_dict.update(tab_evaluation._widgets)
    widget_dict.update(tab_visualization._widgets)

    # Add component widgets
    widget_dict.update(stack_manager._widgets)
    widget_dict.update(action_buttons._widgets)
    widget_dict.update(status_display._widgets)

    return dashboard, widget_dict


def create_complete_interface():
    """
    Create complete interface with dashboard and results area.

    Returns:
        tuple: (complete_ui, widget_dict)
    """

    # Create dashboard
    dashboard, widget_dict = create_unified_dashboard()

    # Create results display
    results_display = create_results_display()

    # Add results widgets to dictionary
    widget_dict.update(results_display._widgets)

    # Combine dashboard and results
    complete_ui = widgets.VBox([
        widgets.HTML("<h1 style='text-align: center; color: #1976D2;'>"
                    "RIS Probe-Based Control - PhD Research Dashboard"
                    "</h1>"),
        widgets.HTML("<hr style='margin: 10px 0;'>"),
        dashboard,
        widgets.HTML("<hr style='margin: 20px 0;'>"),
        results_display
    ])

    return complete_ui, widget_dict


def get_widget_values(widget_dict):
    """
    Extract current values from all widgets.

    Args:
        widget_dict: Dictionary of widgets

    Returns:
        dict: Configuration dictionary with all current values
    """

    config = {}

    # System parameters
    config['N'] = widget_dict['N'].value
    config['K'] = widget_dict['K'].value
    config['M'] = widget_dict['M'].value
    config['P_tx'] = widget_dict['P_tx'].value
    config['sigma_h_sq'] = widget_dict['sigma_h_sq'].value
    config['sigma_g_sq'] = widget_dict['sigma_g_sq'].value
    config['phase_mode'] = widget_dict['phase_mode'].value
    config['phase_bits'] = widget_dict['phase_bits'].value
    config['probe_category'] = widget_dict['probe_category'].value
    config['probe_type'] = widget_dict['probe_type'].value

    # Physics & Realism (Phase 1)
    config['channel_source'] = widget_dict['channel_source'].value
    config['realism_profile'] = widget_dict['realism_profile'].value
    config['use_custom_impairments'] = widget_dict['use_custom_impairments'].value

    if config['use_custom_impairments']:
        config['custom_impairments_config'] = {
            'csi_error': {
                'enabled': True,
                'error_variance_db': widget_dict['csi_error_db'].value
            },
            'channel_aging': {
                'enabled': True,
                'doppler_hz': widget_dict['doppler_hz'].value,
                'feedback_delay_ms': 20  # Fixed for now
            },
            'phase_quantization': {
                'enabled': True,
                'phase_bits': widget_dict['phase_bits_hw'].value
            },
            'quantization': {
                'enabled': True,
                'adc_bits': widget_dict['adc_bits'].value
            },
            'amplitude_control': {'enabled': False},
            'mutual_coupling': {'enabled': False}
        }
    else:
        config['custom_impairments_config'] = None

    # Model parameters (placeholder - will be populated when tab is complete)
    # config['model_preset'] = widget_dict.get('model_preset', widgets.Widget()).value
    # config['num_layers'] = widget_dict.get('num_layers', widgets.Widget()).value
    # ... etc

    # Training parameters (placeholder)
    # config['n_train'] = widget_dict.get('n_train', widgets.Widget()).value
    # ... etc

    # Evaluation parameters (placeholder)
    # config['top_m_values'] = list(widget_dict.get('top_m_values', widgets.Widget()).value)
    # ... etc

    # Visualization parameters (placeholder)
    # config['selected_plots'] = list(widget_dict.get('selected_plots', widgets.Widget()).value)
    # ... etc

    return config


def print_dashboard_info():
    """Print information about the dashboard structure."""

    print("="*70)
    print("RIS DASHBOARD - CLEAN ARCHITECTURE")
    print("="*70)
    print()
    print("Tab Structure:")
    print("  1. System - Core parameters (N, K, M, probes)")
    print("  2. Physics & Realism - Channel sources and impairments")
    print("  3. Model - Architecture configuration")
    print("  4. Training - Training hyperparameters")
    print("  5. Evaluation - Metrics and comparison")
    print("  6. Visualization - Plot selection and settings")
    print()
    print("Components:")
    print("  - Stack Manager (below tabs)")
    print("  - Action Buttons (run, save, load)")
    print("  - Status Display (progress, metrics, logs)")
    print("  - Results Display (summary, plots, export)")
    print()
    print("="*70)
'''

    with open("dashboard/main.py", "w", encoding='utf-8') as f:
        f.write(dashboard_main_content)
    print("[OK] Created: dashboard/main.py")

    # ========================================================================
    # Update dashboard/__init__.py
    # ========================================================================

    dashboard_init_content = '''"""
Dashboard Module
================
Clean, modular dashboard architecture.
"""

from dashboard.main import (
    create_unified_dashboard,
    create_complete_interface,
    get_widget_values,
    print_dashboard_info
)

__all__ = [
    'create_unified_dashboard',
    'create_complete_interface',
    'get_widget_values',
    'print_dashboard_info'
]
'''

    with open("dashboard/__init__.py", "w", encoding='utf-8') as f:
        f.write(dashboard_init_content)
    print("[OK] Updated: dashboard/__init__.py")

    # ========================================================================
    # Create Quick Test Notebook Cell
    # ========================================================================

    test_notebook_content = '''"""
Quick Test - New Dashboard Architecture
========================================
Run this in a notebook cell to test the new clean architecture.
"""

# Test the new dashboard
from dashboard import create_complete_interface, print_dashboard_info

# Print info
print_dashboard_info()

# Create interface
ui, widgets = create_complete_interface()

# Display
display(ui)

# Test getting current configuration
print("\\nCurrent Configuration:")
print("="*70)
from dashboard.main import get_widget_values
config = get_widget_values(widgets)
for key, value in config.items():
    print(f"{key:25s}: {value}")
'''

    with open("test_new_dashboard.py", "w", encoding='utf-8') as f:
        f.write(test_notebook_content)
    print("[OK] Created: test_new_dashboard.py")

    print()
    print("="*70)
    print("DASHBOARD ORCHESTRATOR CREATED SUCCESSFULLY")
    print("="*70)
    print()
    print("Next: Run Stage 5 to complete remaining tab files...")

# Run it
create_dashboard_orchestrator()

Creating dashboard orchestrator...

[OK] Created: dashboard/main.py
[OK] Updated: dashboard/__init__.py
[OK] Created: test_new_dashboard.py

DASHBOARD ORCHESTRATOR CREATED SUCCESSFULLY

Next: Run Stage 5 to complete remaining tab files...


In [7]:
# ============================================================================
# STAGE 5: COMPLETE REMAINING TABS
# ============================================================================

from pathlib import Path

def create_remaining_tabs():
    """Complete the remaining tab files (Model, Training, Eval, Viz)."""

    print("Creating remaining tab files...\n")

    # ========================================================================
    # TAB 3: MODEL ARCHITECTURE
    # ========================================================================

    tab_model_content = '''"""
Tab 3: Model Architecture Configuration
========================================
Model presets, layer configuration, transfer learning.
"""

import ipywidgets as widgets
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '../..')))

try:
    from model_registry import list_models
except ImportError:
    def list_models():
        return ['Baseline_MLP', 'Deep_MLP', 'Custom']


def create_model_tab():
    """Create Model Architecture tab."""

    # Model preset dropdown
    model_options = [
        '━━ Standard MLPs ━━',
        'Baseline_MLP', 'Deep_MLP', 'Tiny_MLP', 'Wide_Deep',
        '━━ Learnable M Selection ━━',
        'LearnedTopK_MLP', 'Attention_MLP', 'Gumbel_MLP', 'RL_MLP',
        '━━ Research Architectures ━━',
        'ResNet_Style', 'Pyramid', 'Hourglass', 'PhD_Custom_1', 'PhD_Custom_2',
        '━━ Custom ━━',
        'Custom'
    ]

    widget_model_preset = widgets.Dropdown(
        options=model_options,
        value='Baseline_MLP',
        description='Model preset:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    # Transfer learning
    widget_transfer_source = widgets.Dropdown(
        options=['None'],
        value='None',
        description='Transfer from:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px'),
        tooltip='Initialize weights from pre-trained model',
        disabled=True
    )

    # Custom architecture controls
    widget_num_layers = widgets.IntSlider(
        value=3, min=1, max=10, step=1,
        description='Number of layers:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px'),
        disabled=True
    )

    # Layer sizes container (dynamically generated)
    widget_layer_sizes_container = widgets.VBox([])

    def update_layer_size_inputs(change=None):
        """Update layer size input widgets based on num_layers."""
        num = widget_num_layers.value
        children = []
        for i in range(num):
            size_widget = widgets.IntText(
                value=512 // (2**i),  # Default: 512, 256, 128, ...
                description=f'Layer {i+1} size:',
                style={'description_width': '180px'},
                layout=widgets.Layout(width='500px')
            )
            children.append(size_widget)
        widget_layer_sizes_container.children = children

    widget_num_layers.observe(update_layer_size_inputs, 'value')
    update_layer_size_inputs()  # Initialize

    # Regularization
    widget_dropout_prob = widgets.FloatSlider(
        value=0.1, min=0.0, max=0.8, step=0.05,
        description='Dropout probability:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    widget_use_batch_norm = widgets.Checkbox(
        value=True,
        description='Use Batch Normalization',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    # Advanced settings
    widget_activation_function = widgets.Dropdown(
        options=['ReLU', 'LeakyReLU', 'GELU', 'ELU', 'Tanh'],
        value='ReLU',
        description='Activation function:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    widget_weight_init = widgets.Dropdown(
        options=['xavier_uniform', 'xavier_normal', 'kaiming_uniform', 'kaiming_normal'],
        value='xavier_uniform',
        description='Weight initialization:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    # Parameter count display
    widget_param_count_display = widgets.HTML(
        value="<b>Estimated parameters:</b> Calculating...",
        layout=widgets.Layout(width='500px')
    )

    # Enable/disable custom controls based on preset
    def toggle_custom_controls(change):
        is_custom = (change['new'] == 'Custom')
        widget_num_layers.disabled = not is_custom
        for child in widget_layer_sizes_container.children:
            child.disabled = not is_custom

    widget_model_preset.observe(toggle_custom_controls, 'value')

    # Layout
    tab_layout = widgets.VBox([
        widgets.HTML("<h3>Model Architecture & Transfer Learning</h3>"),
        widgets.HTML("<h4>Model Selection</h4>"),
        widget_model_preset,
        widget_transfer_source,
        widgets.HTML("<hr style='margin: 15px 0;'>"),
        widgets.HTML("<h4>Custom Architecture (only if 'Custom' selected above)</h4>"),
        widget_num_layers,
        widget_layer_sizes_container,
        widgets.HTML("<hr style='margin: 15px 0;'>"),
        widgets.HTML("<h4>Regularization</h4>"),
        widget_dropout_prob,
        widget_use_batch_norm,
        widgets.HTML("<h4 style='margin-top: 20px;'>Advanced Settings</h4>"),
        widget_activation_function,
        widget_weight_init,
        widgets.HTML("<hr style='margin: 15px 0;'>"),
        widget_param_count_display
    ], layout=widgets.Layout(padding='20px'))

    # Store widget references
    tab_layout._widgets = {
        'model_preset': widget_model_preset,
        'transfer_source': widget_transfer_source,
        'num_layers': widget_num_layers,
        'layer_sizes_container': widget_layer_sizes_container,
        'dropout_prob': widget_dropout_prob,
        'use_batch_norm': widget_use_batch_norm,
        'activation_function': widget_activation_function,
        'weight_init': widget_weight_init,
        'param_count_display': widget_param_count_display
    }

    return tab_layout
'''

    with open("dashboard/tabs/tab_model.py", "w", encoding='utf-8') as f:
        f.write(tab_model_content)
    print("[OK] Created: dashboard/tabs/tab_model.py")

    # ========================================================================
    # TAB 4: TRAINING CONFIGURATION
    # ========================================================================

    tab_training_content = '''"""
Tab 4: Training Configuration
==============================
Dataset sizes, batch size, optimizer, learning rate, scheduler.
"""

import ipywidgets as widgets


def create_training_tab():
    """Create Training Configuration tab."""

    # Dataset parameters
    widget_n_train = widgets.IntText(
        value=50000,
        description='Training samples:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    widget_n_val = widgets.IntText(
        value=5000,
        description='Validation samples:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    widget_n_test = widgets.IntText(
        value=5000,
        description='Test samples:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    widget_seed = widgets.IntText(
        value=42,
        description='Random seed:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    # Data preprocessing
    widget_normalize_input = widgets.Checkbox(
        value=True,
        description='Normalize input',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    widget_normalization_type = widgets.Dropdown(
        options=['mean', 'std', 'log'],
        value='mean',
        description='Normalization type:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    # Training hyperparameters
    widget_batch_size = widgets.Dropdown(
        options=[32, 64, 128, 256, 512],
        value=128,
        description='Batch size:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    widget_learning_rate = widgets.FloatLogSlider(
        value=1e-3, min=-5, max=-1, step=0.1,
        description='Learning rate:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px'),
        readout_format='.1e'
    )

    widget_weight_decay = widgets.FloatLogSlider(
        value=1e-4, min=-6, max=-2, step=0.1,
        description='Weight decay:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px'),
        readout_format='.1e'
    )

    widget_n_epochs = widgets.IntSlider(
        value=50, min=1, max=500, step=1,
        description='Max epochs:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    widget_early_stop_patience = widgets.IntSlider(
        value=10, min=1, max=50, step=1,
        description='Early stop patience:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    # Optimizer and scheduler
    widget_optimizer = widgets.Dropdown(
        options=['Adam', 'AdamW', 'SGD', 'RMSprop'],
        value='Adam',
        description='Optimizer:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    widget_scheduler = widgets.Dropdown(
        options=['ReduceLROnPlateau', 'CosineAnnealing', 'StepLR', 'None'],
        value='ReduceLROnPlateau',
        description='LR scheduler:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    # Layout
    tab_layout = widgets.VBox([
        widgets.HTML("<h3>Training Configuration</h3>"),
        widgets.HTML("<h4>Dataset Sizes</h4>"),
        widget_n_train,
        widget_n_val,
        widget_n_test,
        widget_seed,
        widgets.HTML("<h4 style='margin-top: 20px;'>Data Preprocessing</h4>"),
        widget_normalize_input,
        widget_normalization_type,
        widgets.HTML("<h4 style='margin-top: 20px;'>Training Hyperparameters</h4>"),
        widget_batch_size,
        widget_learning_rate,
        widget_weight_decay,
        widget_n_epochs,
        widget_early_stop_patience,
        widgets.HTML("<h4 style='margin-top: 20px;'>Optimizer & Scheduler</h4>"),
        widget_optimizer,
        widget_scheduler
    ], layout=widgets.Layout(padding='20px'))

    # Store widget references
    tab_layout._widgets = {
        'n_train': widget_n_train,
        'n_val': widget_n_val,
        'n_test': widget_n_test,
        'seed': widget_seed,
        'normalize_input': widget_normalize_input,
        'normalization_type': widget_normalization_type,
        'batch_size': widget_batch_size,
        'learning_rate': widget_learning_rate,
        'weight_decay': widget_weight_decay,
        'n_epochs': widget_n_epochs,
        'early_stop_patience': widget_early_stop_patience,
        'optimizer': widget_optimizer,
        'scheduler': widget_scheduler
    }

    return tab_layout
'''

    with open("dashboard/tabs/tab_training.py", "w", encoding='utf-8') as f:
        f.write(tab_training_content)
    print("[OK] Created: dashboard/tabs/tab_training.py")

    # ========================================================================
    # TAB 5: EVALUATION
    # ========================================================================

    tab_evaluation_content = '''"""
Tab 5: Evaluation & Comparison
===============================
Top-m evaluation, model comparison, multi-seed runs.
"""

import ipywidgets as widgets


def create_evaluation_tab():
    """Create Evaluation & Comparison tab."""

    # Top-m evaluation
    widget_top_m_values = widgets.SelectMultiple(
        options=[1, 2, 4, 8, 16, 32],
        value=[1, 2, 4, 8],
        description='Top-m values:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px', height='120px'),
        tooltip='Select which top-m values to evaluate'
    )

    # Model comparison
    widget_compare_multiple_models = widgets.Checkbox(
        value=False,
        description='Compare multiple models',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    widget_models_to_compare = widgets.SelectMultiple(
        options=['Baseline_MLP', 'Deep_MLP', 'Tiny_MLP'],
        value=[],
        description='Models to compare:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px', height='150px'),
        disabled=True,
        tooltip='Select 2+ models for comparison'
    )

    def toggle_model_comparison(change):
        widget_models_to_compare.disabled = not change['new']

    widget_compare_multiple_models.observe(toggle_model_comparison, 'value')

    # Multi-seed runs
    widget_multi_seed_runs = widgets.Checkbox(
        value=False,
        description='Multi-seed runs',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    widget_num_seeds = widgets.IntSlider(
        value=3, min=1, max=10, step=1,
        description='Number of seeds:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px'),
        disabled=True
    )

    def toggle_multi_seed(change):
        widget_num_seeds.disabled = not change['new']

    widget_multi_seed_runs.observe(toggle_multi_seed, 'value')

    # Statistical analysis
    widget_compute_confidence_intervals = widgets.Checkbox(
        value=False,
        description='Compute confidence intervals',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    # Layout
    tab_layout = widgets.VBox([
        widgets.HTML("<h3>Evaluation & Comparison</h3>"),
        widgets.HTML("<h4>Top-m Evaluation</h4>"),
        widget_top_m_values,
        widgets.HTML("<h4 style='margin-top: 20px;'>Model Comparison</h4>"),
        widget_compare_multiple_models,
        widget_models_to_compare,
        widgets.HTML("<h4 style='margin-top: 20px;'>Statistical Validation</h4>"),
        widget_multi_seed_runs,
        widget_num_seeds,
        widget_compute_confidence_intervals
    ], layout=widgets.Layout(padding='20px'))

    # Store widget references
    tab_layout._widgets = {
        'top_m_values': widget_top_m_values,
        'compare_multiple_models': widget_compare_multiple_models,
        'models_to_compare': widget_models_to_compare,
        'multi_seed_runs': widget_multi_seed_runs,
        'num_seeds': widget_num_seeds,
        'compute_confidence_intervals': widget_compute_confidence_intervals
    }

    return tab_layout
'''

    with open("dashboard/tabs/tab_evaluation.py", "w", encoding='utf-8') as f:
        f.write(tab_evaluation_content)
    print("[OK] Created: dashboard/tabs/tab_evaluation.py")

    # ========================================================================
    # TAB 6: VISUALIZATION
    # ========================================================================

    tab_visualization_content = '''"""
Tab 6: Visualization Control
=============================
Plot selection, output format, styling options.
"""

import ipywidgets as widgets
from dashboard.components.buttons import create_results_buttons


def create_visualization_tab():
    """Create Visualization Control tab."""

    # Plot type selection
    ALL_PLOT_TYPES = [
        'training_curves', 'eta_distribution', 'cdf',
        'top_m_comparison', 'top_m_efficiency', 'baseline_comparison',
        'violin', 'box', 'scatter',
        'heatmap',
        'eta_vs_M', 'eta_vs_K', 'eta_vs_N'
    ]

    widget_selected_plots = widgets.SelectMultiple(
        options=ALL_PLOT_TYPES,
        value=['training_curves', 'eta_distribution', 'top_m_comparison', 'top_m_efficiency'],
        description='Select plots:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px', height='280px')
    )

    # Output settings
    widget_figure_format = widgets.Dropdown(
        options=['png', 'pdf', 'svg'],
        value='png',
        description='Figure format:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    widget_dpi = widgets.IntSlider(
        value=150, min=72, max=300, step=6,
        description='DPI:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    # Styling
    widget_color_palette = widgets.Dropdown(
        options=['viridis', 'plasma', 'inferno', 'magma', 'cividis', 'seaborn'],
        value='viridis',
        description='Color palette:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    # Save options
    widget_save_plots = widgets.Checkbox(
        value=True,
        description='Save plots to disk',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    widget_output_dir = widgets.Text(
        value='results/',
        description='Output directory:',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )

    # Results management buttons
    results_buttons = create_results_buttons()

    # Layout
    tab_layout = widgets.VBox([
        widgets.HTML("<h3>Visualization Control</h3>"),
        widgets.HTML("<h4>Plot Mode</h4>"),
        results_buttons,
        widgets.HTML("<hr style='margin: 15px 0;'>"),
        widgets.HTML("<h4>Plot Selection</h4>"),
        widget_selected_plots,
        widgets.HTML("<h4 style='margin-top: 20px;'>Output Settings</h4>"),
        widget_figure_format,
        widget_dpi,
        widgets.HTML("<h4 style='margin-top: 20px;'>Styling</h4>"),
        widget_color_palette,
        widgets.HTML("<h4 style='margin-top: 20px;'>Save Options</h4>"),
        widget_save_plots,
        widget_output_dir
    ], layout=widgets.Layout(padding='20px'))

    # Store widget references
    tab_layout._widgets = {
        'selected_plots': widget_selected_plots,
        'figure_format': widget_figure_format,
        'dpi': widget_dpi,
        'color_palette': widget_color_palette,
        'save_plots': widget_save_plots,
        'output_dir': widget_output_dir
    }
    tab_layout._widgets.update(results_buttons._widgets)

    return tab_layout
'''

    with open("dashboard/tabs/tab_visualization.py", "w", encoding='utf-8') as f:
        f.write(tab_visualization_content)
    print("[OK] Created: dashboard/tabs/tab_visualization.py")

    # Update tabs/__init__.py
    tabs_init_content = '''"""
Dashboard Tabs Module
=====================
Each tab in a separate, maintainable file.
"""

from dashboard.tabs.tab_system import create_system_tab
from dashboard.tabs.tab_physics import create_physics_tab
from dashboard.tabs.tab_model import create_model_tab
from dashboard.tabs.tab_training import create_training_tab
from dashboard.tabs.tab_evaluation import create_evaluation_tab
from dashboard.tabs.tab_visualization import create_visualization_tab

__all__ = [
    'create_system_tab',
    'create_physics_tab',
    'create_model_tab',
    'create_training_tab',
    'create_evaluation_tab',
    'create_visualization_tab'
]
'''

    with open("dashboard/tabs/__init__.py", "w", encoding='utf-8') as f:
        f.write(tabs_init_content)
    print("[OK] Updated: dashboard/tabs/__init__.py")

    print()
    print("="*70)
    print("ALL TAB FILES COMPLETED SUCCESSFULLY")
    print("="*70)
    print()
    print("Next: Run Stage 6 for testing and validation...")

# Run it
create_remaining_tabs()

Creating remaining tab files...

[OK] Created: dashboard/tabs/tab_model.py
[OK] Created: dashboard/tabs/tab_training.py
[OK] Created: dashboard/tabs/tab_evaluation.py
[OK] Created: dashboard/tabs/tab_visualization.py
[OK] Updated: dashboard/tabs/__init__.py

ALL TAB FILES COMPLETED SUCCESSFULLY

Next: Run Stage 6 for testing and validation...


In [8]:
# ============================================================================
# STAGE 6: TESTING, VALIDATION & MIGRATION
# ============================================================================

from pathlib import Path

def create_testing_and_migration():
    """Create testing scripts and migration guide."""

    print("Creating testing and migration tools...\n")

    # ========================================================================
    # COMPREHENSIVE TEST SCRIPT
    # ========================================================================

    test_complete_content = '''"""
Comprehensive Test Script for New Dashboard Architecture
=========================================================
Tests all components to ensure everything works correctly.
"""

def test_complete_dashboard():
    """Run complete test suite for new dashboard."""

    print("="*70)
    print("COMPREHENSIVE DASHBOARD TEST SUITE")
    print("="*70)
    print()

    # Test 1: Import all modules
    print("[TEST 1] Importing all modules...")
    try:
        from dashboard.tabs import (
            create_system_tab,
            create_physics_tab,
            create_model_tab,
            create_training_tab,
            create_evaluation_tab,
            create_visualization_tab
        )
        from dashboard.components import (
            create_stack_manager,
            create_action_buttons,
            create_status_display,
            create_results_display
        )
        from dashboard.main import (
            create_unified_dashboard,
            create_complete_interface,
            get_widget_values
        )
        print("[OK] All modules imported successfully")
    except Exception as e:
        print(f"[FAIL] Import error: {e}")
        return False

    # Test 2: Create individual tabs
    print("\\n[TEST 2] Creating individual tabs...")
    try:
        tab_system = create_system_tab()
        print("  [OK] System tab created")

        tab_physics = create_physics_tab()
        print("  [OK] Physics tab created")

        tab_model = create_model_tab()
        print("  [OK] Model tab created")

        tab_training = create_training_tab()
        print("  [OK] Training tab created")

        tab_evaluation = create_evaluation_tab()
        print("  [OK] Evaluation tab created")

        tab_visualization = create_visualization_tab()
        print("  [OK] Visualization tab created")
    except Exception as e:
        print(f"  [FAIL] Tab creation error: {e}")
        import traceback
        traceback.print_exc()
        return False

    # Test 3: Create components
    print("\\n[TEST 3] Creating components...")
    try:
        stack_manager = create_stack_manager()
        print("  [OK] Stack manager created")

        action_buttons = create_action_buttons()
        print("  [OK] Action buttons created")

        status_display = create_status_display()
        print("  [OK] Status display created")

        results_display = create_results_display()
        print("  [OK] Results display created")
    except Exception as e:
        print(f"  [FAIL] Component creation error: {e}")
        import traceback
        traceback.print_exc()
        return False

    # Test 4: Create unified dashboard
    print("\\n[TEST 4] Creating unified dashboard...")
    try:
        dashboard, widget_dict = create_unified_dashboard()
        print(f"  [OK] Dashboard created with {len(widget_dict)} widgets")
    except Exception as e:
        print(f"  [FAIL] Dashboard creation error: {e}")
        import traceback
        traceback.print_exc()
        return False

    # Test 5: Create complete interface
    print("\\n[TEST 5] Creating complete interface...")
    try:
        ui, widgets = create_complete_interface()
        print(f"  [OK] Complete interface created")
    except Exception as e:
        print(f"  [FAIL] Interface creation error: {e}")
        import traceback
        traceback.print_exc()
        return False

    # Test 6: Extract configuration
    print("\\n[TEST 6] Extracting configuration...")
    try:
        config = get_widget_values(widgets)
        print(f"  [OK] Configuration extracted")
        print(f"      N={config.get('N')}, K={config.get('K')}, M={config.get('M')}")
        print(f"      Channel source: {config.get('channel_source')}")
        print(f"      Realism profile: {config.get('realism_profile')}")
    except Exception as e:
        print(f"  [FAIL] Configuration extraction error: {e}")
        import traceback
        traceback.print_exc()
        return False

    # Test 7: Widget access
    print("\\n[TEST 7] Testing widget access...")
    try:
        # Test system widgets
        assert 'N' in widgets, "N widget not found"
        assert 'K' in widgets, "K widget not found"
        assert 'M' in widgets, "M widget not found"

        # Test physics widgets
        assert 'channel_source' in widgets, "channel_source widget not found"
        assert 'realism_profile' in widgets, "realism_profile widget not found"

        # Test buttons
        assert 'button_run_experiment' in widgets, "Run button not found"
        assert 'button_add_to_stack' in widgets, "Add to stack button not found"

        print("  [OK] All key widgets accessible")
    except AssertionError as e:
        print(f"  [FAIL] Widget access error: {e}")
        return False

    # Test 8: Physics integration
    print("\\n[TEST 8] Testing Phase 1 physics integration...")
    try:
        from physics import (
            list_available_sources,
            list_profiles,
            create_pipeline_from_profile
        )

        sources = list_available_sources()
        print(f"  [OK] {len(sources)} channel sources available")

        profiles = list_profiles()
        print(f"  [OK] {len(profiles)} realism profiles available")

        pipeline = create_pipeline_from_profile('moderate_impairments')
        print(f"  [OK] Impairment pipeline created")
    except Exception as e:
        print(f"  [FAIL] Physics integration error: {e}")
        import traceback
        traceback.print_exc()
        return False

    print()
    print("="*70)
    print("ALL TESTS PASSED!")
    print("="*70)
    print()
    print("The new dashboard architecture is ready to use.")
    print()
    print("To display the dashboard:")
    print("  from dashboard import create_complete_interface")
    print("  ui, widgets = create_complete_interface()")
    print("  display(ui)")

    return True

# Run test
if __name__ == "__main__":
    test_complete_dashboard()
'''

    with open("test_complete_dashboard.py", "w", encoding='utf-8') as f:
        f.write(test_complete_content)
    print("[OK] Created: test_complete_dashboard.py")

    # ========================================================================
    # MIGRATION GUIDE
    # ========================================================================

    migration_guide_content = '''# Migration Guide: Old → New Dashboard Architecture

## Overview

This guide helps you transition from the monolithic `widgets.py` to the clean, modular architecture.

## Architecture Comparison

### Old Structure (Monolithic)
```
Session5_clone/
├── widgets.py                    # Everything in one file (2000+ lines)
├── callbacks.py                  # Event handlers
├── experiment_runner.py          # Experiment logic
├── config.py                     # Configuration
└── [other scattered files]
```

### New Structure (Modular)
```
Session5_clone/
├── dashboard/
│   ├── tabs/                    # Each tab = separate file
│   │   ├── tab_system.py
│   │   ├── tab_physics.py
│   │   ├── tab_model.py
│   │   ├── tab_training.py
│   │   ├── tab_evaluation.py
│   │   └── tab_visualization.py
│   ├── components/              # Reusable components
│   │   ├── stack_manager.py
│   │   ├── buttons.py
│   │   ├── status_display.py
│   │   └── results_display.py
│   ├── main.py                  # Dashboard orchestrator
│   └── callbacks.py             # Event handlers (same)
├── physics/                     # Phase 1 physics module
├── models/                      # Neural network models
├── data/                        # Data generation
└── [other organized folders]
```

## Migration Steps

### Step 1: Test New Dashboard (NO CHANGES TO OLD CODE YET)
```python
# In a new notebook cell:
exec(open('test_complete_dashboard.py').read())
```

Expected output: All tests pass ✓

### Step 2: Display New Dashboard (Side-by-Side Comparison)
```python
# Test new dashboard
from dashboard import create_complete_interface
ui_new, widgets_new = create_complete_interface()
display(ui_new)

# Old dashboard still works
# (your existing notebook code unchanged)
```

### Step 3: Update Your Notebook (When Ready)

**OLD CODE (in your notebook):**
```python
from widgets import create_unified_dashboard, get_all_widgets
from callbacks import setup_callbacks

dashboard = create_unified_dashboard()
widgets = get_all_widgets()
setup_callbacks(widgets)
display(dashboard)
```

**NEW CODE (replace with this):**
```python
from dashboard import create_complete_interface
from callbacks import setup_callbacks  # Same file, works as-is

ui, widgets = create_complete_interface()
setup_callbacks(widgets)  # Same callbacks work!
display(ui)
```

### Step 4: Update Callbacks (Minor Changes)

Your `callbacks.py` should work mostly as-is. Only change needed:

**OLD:**
```python
def config_to_dict(wd):
    config = {
        'N': wd['N'].value,
        'K': wd['K'].value,
        # ... etc
    }
```

**NEW:**
```python
from dashboard.main import get_widget_values

def config_to_dict(wd):
    # Option 1: Use helper function (recommended)
    config = get_widget_values(wd)

    # Option 2: Manual (same as before, still works)
    config = {
        'N': wd['N'].value,
        'K': wd['K'].value,
        # ... etc
    }
```

The `get_widget_values()` helper is optional but recommended - it extracts all config automatically.

## What Stays The Same

✅ **callbacks.py** - Works as-is (widget dict keys unchanged)
✅ **experiment_runner.py** - No changes needed
✅ **All your existing saved configs** - Compatible
✅ **All your experiment results** - Compatible

## What's Different

🔄 **Tab organization** - Cleaner, each in separate file
🔄 **Import statements** - `from dashboard import ...` instead of `from widgets import ...`
✅ **Everything else** - Same functionality

## Testing Checklist

Before switching completely:

- [ ] Run `test_complete_dashboard.py` - all tests pass
- [ ] Display new dashboard - UI renders correctly
- [ ] Click through all 6 tabs - no errors
- [ ] Test "Add to Stack" - works
- [ ] Extract config with `get_widget_values()` - returns dict
- [ ] Physics profile selection - updates info box
- [ ] Custom impairments toggle - shows/hides advanced settings

## Rollback Plan

If anything breaks, you can instantly rollback:
```python
# Use old dashboard (no changes needed to your code)
from widgets import create_unified_dashboard, get_all_widgets
dashboard = create_unified_dashboard()
widgets = get_all_widgets()
display(dashboard)
```

Your old `widgets.py` file is untouched and still works!

## Benefits of New Architecture

1. **Maintainability** - Each tab ~200 lines vs. one 2000+ line file
2. **Collaboration** - Multiple people can edit different tabs
3. **Testing** - Test individual tabs/components
4. **Reusability** - Components used across multiple dashboards
5. **Scalability** - Easy to add new tabs/features
6. **Professional** - Industry-standard Python package structure

## File Mapping Reference

| Old Location | New Location |
|--------------|-------------|
| `widgets.py` (Tab 1) | `dashboard/tabs/tab_system.py` |
| `widgets.py` (Tab 2 - Physics) | `dashboard/tabs/tab_physics.py` |
| `widgets.py` (Tab 3) | `dashboard/tabs/tab_model.py` |
| `widgets.py` (Tab 4) | `dashboard/tabs/tab_training.py` |
| `widgets.py` (Tab 5) | `dashboard/tabs/tab_evaluation.py` |
| `widgets.py` (Tab 6) | `dashboard/tabs/tab_visualization.py` |
| `widgets.py` (Stack) | `dashboard/components/stack_manager.py` |
| `widgets.py` (Buttons) | `dashboard/components/buttons.py` |
| `widgets.py` (Status) | `dashboard/components/status_display.py` |
| `widgets.py` (Results) | `dashboard/components/results_display.py` |
| `widgets.py` (Main) | `dashboard/main.py` |

## Next Steps

1. **Test** - Run `test_complete_dashboard.py`
2. **Compare** - Display both old and new dashboards side-by-side
3. **Migrate** - Update your notebook when ready
4. **Enjoy** - Clean, maintainable code structure!

## Questions?

- Check `test_complete_dashboard.py` for working examples
- Review individual tab files in `dashboard/tabs/`
- Look at `dashboard/main.py` to see how it all connects

---

**Remember:** The old dashboard still works! Take your time to test the new one.
'''

    with open("docs/MIGRATION_GUIDE.md", "w", encoding='utf-8') as f:
        f.write(migration_guide_content)
    print("[OK] Created: docs/MIGRATION_GUIDE.md")

    # ========================================================================
    # QUICK START GUIDE
    # ========================================================================

    quickstart_content = '''# Quick Start: New Dashboard Architecture

## Immediate Usage (No Migration Needed)

### Option 1: Test in Isolation
```python
# In a new notebook cell - test without affecting your current setup
exec(open('test_complete_dashboard.py').read())
```

This runs all tests and shows you the new dashboard is working.

### Option 2: Display New Dashboard
```python
from dashboard import create_complete_interface

# Create interface
ui, widgets = create_complete_interface()

# Display it
display(ui)

# Get current configuration
from dashboard.main import get_widget_values
config = get_widget_values(widgets)
print(config)
```

### Option 3: Use With Existing Callbacks
```python
from dashboard import create_complete_interface
from callbacks import setup_callbacks  # Your existing callbacks

# Create interface
ui, widgets = create_complete_interface()

# Setup callbacks (your existing code works!)
setup_callbacks(widgets)

# Display
display(ui)
```

## Key Features

### 6 Clean Tabs

1. **System** - N, K, M, probes, channel params
2. **Physics & Realism** - Channel sources, impairment profiles
3. **Model** - Architecture, transfer learning
4. **Training** - Hyperparameters, optimizer
5. **Evaluation** - Top-m, comparison, multi-seed
6. **Visualization** - Plot selection, styling

### Components Always Visible

- **Stack Manager** (below tabs) - Add/remove experiments
- **Action Buttons** - Run, save, load, reset
- **Status Display** - Progress, metrics, logs
- **Results Display** - Summary, plots, export

## File Organization
```
dashboard/
├── tabs/              # Each tab = separate file
│   ├── tab_system.py
│   ├── tab_physics.py
│   ├── tab_model.py
│   ├── tab_training.py
│   ├── tab_evaluation.py
│   └── tab_visualization.py
│
├── components/        # Reusable UI components
│   ├── stack_manager.py
│   ├── buttons.py
│   ├── status_display.py
│   └── results_display.py
│
└── main.py           # Orchestrator (brings it all together)
```

## Common Tasks

### Get All Widget Values
```python
from dashboard.main import get_widget_values

config = get_widget_values(widgets)
# Returns dict with all parameters
```

### Access Individual Widgets
```python
# All widgets accessible via dictionary
widgets['N'].value = 64
widgets['realism_profile'].value = 'moderate_impairments'
widgets['button_run_experiment'].on_click(my_callback)
```

### Modify Tab Content

Each tab file is standalone. Edit any tab without touching others:
```python
# Edit dashboard/tabs/tab_system.py to customize System tab
# Edit dashboard/tabs/tab_physics.py to customize Physics tab
# etc.
```

## Comparison with Old Dashboard

| Feature | Old | New |
|---------|-----|-----|
| Total files | 1 (widgets.py) | 11+ (organized) |
| Lines per file | 2000+ | ~200 each |
| Tab editing | Scroll through one file | Open specific tab file |
| Widget access | Same dict | Same dict ✓ |
| Callbacks | Compatible | Compatible ✓ |
| Configs | Compatible | Compatible ✓ |

## Phase 1 Physics Integration

The new dashboard is fully integrated with Phase 1 physics:
```python
# Automatic physics tracking
widgets['channel_source'].value = 'python_synthetic'
widgets['realism_profile'].value = 'moderate_impairments'

# Configuration includes physics metadata
config = get_widget_values(widgets)
print(config['channel_source'])     # 'python_synthetic'
print(config['realism_profile'])    # 'moderate_impairments'
```

## Testing

Run comprehensive tests:
```python
exec(open('test_complete_dashboard.py').read())
```

Expected output:
```
[TEST 1] Importing all modules... [OK]
[TEST 2] Creating individual tabs... [OK]
[TEST 3] Creating components... [OK]
[TEST 4] Creating unified dashboard... [OK]
[TEST 5] Creating complete interface... [OK]
[TEST 6] Extracting configuration... [OK]
[TEST 7] Testing widget access... [OK]
[TEST 8] Testing Phase 1 physics integration... [OK]

ALL TESTS PASSED!
```

## Need Help?

1. **See working example**: `test_complete_dashboard.py`
2. **Read migration guide**: `docs/MIGRATION_GUIDE.md`
3. **Check individual tabs**: `dashboard/tabs/tab_*.py`
4. **Review main file**: `dashboard/main.py`

---

**The new architecture is ready to use immediately - no migration required to test it!**
'''

    with open("docs/QUICKSTART.md", "w", encoding='utf-8') as f:
        f.write(quickstart_content)
    print("[OK] Created: docs/QUICKSTART.md")

    # ========================================================================
    # CREATE FINAL SUMMARY
    # ========================================================================

    summary_content = '''# 🎉 CLEAN ARCHITECTURE COMPLETE!

## What Was Created

### ✅ Folder Structure
```
Session5_clone/
├── config/              [NEW] Configuration management
├── physics/             [DONE] Phase 1 physics module
├── models/              [NEW] Neural network models
├── data/                [NEW] Data generation
├── training/            [NEW] Training pipeline
├── evaluation/          [NEW] Evaluation & metrics
├── dashboard/           [NEW] Modular dashboard
│   ├── tabs/           6 separate tab files
│   ├── components/     4 reusable components
│   └── main.py         Dashboard orchestrator
├── plotting/            [NEW] Visualization
├── notebooks/           Jupyter notebooks
├── results/             Generated results
├── tests/               Unit tests
└── docs/                Documentation
```

### ✅ Dashboard Tabs (Separate Files)
1. `tab_system.py` - Core system parameters
2. `tab_physics.py` - Physics & realism (Phase 1)
3. `tab_model.py` - Model architecture
4. `tab_training.py` - Training configuration
5. `tab_evaluation.py` - Evaluation & comparison
6. `tab_visualization.py` - Plot control

### ✅ Reusable Components
1. `stack_manager.py` - Experiment stack UI
2. `buttons.py` - Action & export buttons
3. `status_display.py` - Progress & logs
4. `results_display.py` - Results area

### ✅ Documentation
1. `MIGRATION_GUIDE.md` - Complete migration instructions
2. `QUICKSTART.md` - Immediate usage guide
3. `PHASE1_README.md` - Phase 1 physics documentation

### ✅ Testing
1. `test_complete_dashboard.py` - Comprehensive test suite
2. `test_phase1.py` - Phase 1 physics tests

## Current Status

🟢 **All files created and ready to use**
🟢 **Backward compatible** - old code still works
🟢 **Phase 1 physics fully integrated**
🟢 **Professional folder structure**

## Next Steps

### Immediate (Testing - No Risk)
```python
# 1. Test the new architecture
exec(open('test_complete_dashboard.py').read())

# 2. Display new dashboard (doesn't affect old code)
from dashboard import create_complete_interface
ui, widgets = create_complete_interface()
display(ui)
```

### When Ready (Migration)

Follow `docs/MIGRATION_GUIDE.md` to switch your notebook to use the new dashboard.

**Key Point:** Your old `widgets.py` still works! No rush to migrate.

## Benefits

✅ **Maintainability** - 200 lines per file vs. 2000+ in one
✅ **Modularity** - Edit tabs independently
✅ **Scalability** - Easy to add features
✅ **Professional** - Industry-standard structure
✅ **Testable** - Test individual components
✅ **Collaborative** - Multiple people can work on different tabs

## File Comparison

| Aspect | Old Structure | New Structure |
|--------|---------------|---------------|
| Main file | widgets.py (2000+ lines) | 11+ organized files |
| Tab editing | Scroll through one file | Open specific tab file |
| Testing | Hard to isolate | Test each component |
| Adding features | Add to monolith | Create new file |
| Team work | Conflicts | Parallel work |
| Maintenance | Find in 2000 lines | Go to specific file |

## Commands Reference

### Test Everything
```bash
python test_complete_dashboard.py
```

### Test Phase 1 Physics
```bash
python test_phase1.py
```

### View Documentation
```bash
# Migration guide
cat docs/MIGRATION_GUIDE.md

# Quick start
cat docs/QUICKSTART.md

# Phase 1 docs
cat PHASE1_README.md
```

## What Didn't Change

✅ Widget dictionary keys - all the same
✅ Callback signatures - compatible
✅ Configuration format - same
✅ Experiment results - compatible
✅ Saved configs - work as before

## Summary

You now have:
1. ✅ Clean, professional folder structure
2. ✅ Modular dashboard (6 tabs + 4 components)
3. ✅ Phase 1 physics fully integrated
4. ✅ Complete documentation
5. ✅ Comprehensive test suite
6. ✅ 100% backward compatibility

**Your old code still works - test the new architecture risk-free!**

---

Questions? See:
- `docs/QUICKSTART.md` - Get started immediately
- `docs/MIGRATION_GUIDE.md` - Detailed migration steps
- `test_complete_dashboard.py` - Working examples
'''

    with open("ARCHITECTURE_COMPLETE.md", "w", encoding='utf-8') as f:
        f.write(summary_content)
    print("[OK] Created: ARCHITECTURE_COMPLETE.md")

    print()
    print("="*70)
    print("STAGE 6 COMPLETE - ALL SETUP FINISHED!")
    print("="*70)
    print()
    print("📚 Documentation created:")
    print("   - docs/MIGRATION_GUIDE.md")
    print("   - docs/QUICKSTART.md")
    print("   - ARCHITECTURE_COMPLETE.md")
    print()
    print("🧪 Test scripts created:")
    print("   - test_complete_dashboard.py")
    print()
    print("="*70)
    print("NEXT: Test the new architecture!")
    print("="*70)
    print()
    print("Run in a notebook cell:")
    print("  exec(open('test_complete_dashboard.py').read())")
    print()
    print("Or display the new dashboard:")
    print("  from dashboard import create_complete_interface")
    print("  ui, widgets = create_complete_interface()")
    print("  display(ui)")
    print()
    print("See ARCHITECTURE_COMPLETE.md for full summary!")

# Run it
create_testing_and_migration()

Creating testing and migration tools...

[OK] Created: test_complete_dashboard.py
[OK] Created: docs/MIGRATION_GUIDE.md
[OK] Created: docs/QUICKSTART.md
[OK] Created: ARCHITECTURE_COMPLETE.md

STAGE 6 COMPLETE - ALL SETUP FINISHED!

📚 Documentation created:
   - docs/MIGRATION_GUIDE.md
   - docs/QUICKSTART.md
   - ARCHITECTURE_COMPLETE.md

🧪 Test scripts created:
   - test_complete_dashboard.py

NEXT: Test the new architecture!

Run in a notebook cell:
  exec(open('test_complete_dashboard.py').read())

Or display the new dashboard:
  from dashboard import create_complete_interface
  ui, widgets = create_complete_interface()
  display(ui)

See ARCHITECTURE_COMPLETE.md for full summary!


In [9]:
exec(open('test_complete_dashboard.py').read())

COMPREHENSIVE DASHBOARD TEST SUITE

[TEST 1] Importing all modules...
[OK] All modules imported successfully

[TEST 2] Creating individual tabs...
  [OK] System tab created
  [OK] Physics tab created
  [OK] Model tab created
  [OK] Training tab created
  [OK] Evaluation tab created
  [OK] Visualization tab created

[TEST 3] Creating components...
  [OK] Stack manager created
  [OK] Action buttons created
  [OK] Status display created
  [OK] Results display created

[TEST 4] Creating unified dashboard...
  [OK] Dashboard created with 75 widgets

[TEST 5] Creating complete interface...
  [OK] Complete interface created

[TEST 6] Extracting configuration...
  [OK] Configuration extracted
      N=32, K=64, M=8
      Channel source: python_synthetic
      Realism profile: ideal

[TEST 7] Testing widget access...
  [OK] All key widgets accessible

[TEST 8] Testing Phase 1 physics integration...
  [OK] 3 channel sources available
  [OK] 5 realism profiles available
  [OK] Impairment pipeline 

In [10]:
# ============================================================================
# STAGE 7: SAFE CLEANUP - Identify and Remove Unused Files
# ============================================================================

import os
import shutil
from pathlib import Path
from datetime import datetime

def create_cleanup_plan():
    """
    Analyze project and create cleanup plan.
    Shows what will be deleted/archived before doing anything.
    """

    print("="*70)
    print("PROJECT CLEANUP ANALYSIS")
    print("="*70)
    print()

    # ========================================================================
    # STEP 1: Identify files that are NOW OBSOLETE
    # ========================================================================

    obsolete_files = {
        'widgets.py': 'Replaced by dashboard/tabs/* and dashboard/components/*',
        'widgets_system_physics_v2.py': 'Replaced by dashboard/tabs/tab_physics.py',
        'config_updated.py': 'Merge into config/ folder or keep as backup',
        'PHASE1_NOTEBOOK_INTEGRATION.py': 'One-time setup, can archive',
        'test_phase1.py': 'Keep or move to tests/',
        'test_new_dashboard.py': 'Keep or move to tests/',
    }

    print("📋 OBSOLETE FILES (replaced by new architecture):")
    print("-" * 70)

    for file, reason in obsolete_files.items():
        if Path(file).exists():
            size = Path(file).stat().st_size / 1024  # KB
            print(f"  ❌ {file:<40} ({size:.1f} KB)")
            print(f"      Reason: {reason}")
        else:
            print(f"  ⚠️  {file:<40} (NOT FOUND)")

    print()

    # ========================================================================
    # STEP 2: Identify files to KEEP
    # ========================================================================

    keep_files = {
        # Core functionality (still used)
        'config.py': 'Core config (if not migrated to config/ yet)',
        'data_generation.py': 'Keep until migrated to data/',
        'model_registry.py': 'Keep until migrated to models/',
        'experiment_runner.py': 'Keep until migrated to dashboard/',
        'callbacks.py': 'Keep until migrated to dashboard/',

        # Documentation (keep)
        'README.md': 'Main documentation',
        'PHASE1_README.md': 'Phase 1 documentation',
        'PHASE1_INTEGRATION_GUIDE.py': 'Reference documentation',
        'ARCHITECTURE_COMPLETE.md': 'Architecture summary',

        # Tests (keep or move)
        'test_complete_dashboard.py': 'Main test suite',

        # New architecture (all keep)
        'dashboard/': 'New modular dashboard',
        'physics/': 'Phase 1 physics module',
        'config/': 'New config module',
        'models/': 'New models module',
        'data/': 'New data module',
        'docs/': 'Documentation',
    }

    print("✅ FILES TO KEEP (still needed):")
    print("-" * 70)
    for file, reason in keep_files.items():
        print(f"  ✓ {file:<40} - {reason}")

    print()

    # ========================================================================
    # STEP 3: Create backup before cleanup
    # ========================================================================

    print("="*70)
    print("RECOMMENDED CLEANUP PROCEDURE")
    print("="*70)
    print()
    print("Option 1: SAFE ARCHIVE (Recommended)")
    print("  • Create 'archive_old_code/' folder")
    print("  • Move obsolete files there (not delete)")
    print("  • Can restore anytime if needed")
    print()
    print("Option 2: DELETE (Only if confident)")
    print("  • Permanently delete obsolete files")
    print("  • Make sure everything works first!")
    print()

    return obsolete_files


def safe_archive_cleanup():
    """
    SAFE cleanup: Archive old files instead of deleting.
    """

    print("="*70)
    print("SAFE ARCHIVE CLEANUP")
    print("="*70)
    print()

    # Create archive folder with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    archive_dir = Path(f"archive_old_code_{timestamp}")
    archive_dir.mkdir(exist_ok=True)

    print(f"Created archive directory: {archive_dir}/")
    print()

    # Files to archive (not delete!)
    files_to_archive = [
        'widgets.py',
        'widgets_system_physics_v2.py',
        'PHASE1_NOTEBOOK_INTEGRATION.py',
    ]

    archived_count = 0

    for file in files_to_archive:
        filepath = Path(file)
        if filepath.exists():
            # Move to archive
            dest = archive_dir / filepath.name
            shutil.copy2(filepath, dest)
            print(f"  ✓ Archived: {file} -> {dest}")
            archived_count += 1
        else:
            print(f"  ⚠️  Not found: {file}")

    print()
    print(f"Archived {archived_count} files to {archive_dir}/")
    print()
    print("="*70)
    print("NEXT STEPS:")
    print("="*70)
    print()
    print("1. Test your dashboard thoroughly")
    print("2. If everything works, you can delete archived files")
    print("3. Or keep archive folder as backup")
    print()
    print("To restore a file:")
    print(f"  cp {archive_dir}/widgets.py .")
    print()

    return archive_dir


def aggressive_cleanup():
    """
    AGGRESSIVE cleanup: Actually delete obsolete files.

    ⚠️ WARNING: Only run this AFTER testing everything works!
    """

    print()
    print("⚠️  WARNING: AGGRESSIVE CLEANUP MODE")
    print("="*70)
    print()
    print("This will PERMANENTLY DELETE obsolete files!")
    print()

    response = input("Are you SURE everything works? Type 'DELETE' to confirm: ")

    if response != "DELETE":
        print("Cleanup cancelled. Good choice - test more first!")
        return

    print()
    print("Deleting obsolete files...")
    print()

    files_to_delete = [
        'widgets.py',
        'widgets_system_physics_v2.py',
        'PHASE1_NOTEBOOK_INTEGRATION.py',
    ]

    deleted_count = 0

    for file in files_to_delete:
        filepath = Path(file)
        if filepath.exists():
            filepath.unlink()
            print(f"  ❌ Deleted: {file}")
            deleted_count += 1
        else:
            print(f"  ⚠️  Not found: {file}")

    print()
    print(f"Deleted {deleted_count} files")
    print()
    print("✅ Cleanup complete!")


def organize_remaining_files():
    """
    Organize files that haven't been migrated yet.
    """

    print("="*70)
    print("ORGANIZE REMAINING FILES")
    print("="*70)
    print()

    # Files that should be migrated but might not be yet
    files_to_organize = {
        'config.py': 'config/',
        'data_generation.py': 'data/',
        'model_registry.py': 'models/',
        'callbacks.py': 'dashboard/',
        'experiment_runner.py': 'dashboard/',
    }

    print("Files that should be organized:")
    print()

    for file, target_dir in files_to_organize.items():
        filepath = Path(file)
        if filepath.exists():
            target_path = Path(target_dir) / filepath.name

            if target_path.exists():
                print(f"  ✓ {file:<30} Already in {target_dir}")
            else:
                print(f"  ➡️  {file:<30} Should move to {target_dir}")
        else:
            print(f"  ⚠️  {file:<30} Not found")

    print()
    print("To organize these files, run: migrate_remaining_files()")


def migrate_remaining_files():
    """
    Actually move files to their proper locations.
    """

    print("="*70)
    print("MIGRATING REMAINING FILES")
    print("="*70)
    print()

    migrations = {
        'callbacks.py': 'dashboard/callbacks.py',
        'experiment_runner.py': 'dashboard/experiment_runner.py',
    }

    for source, dest in migrations.items():
        source_path = Path(source)
        dest_path = Path(dest)

        if source_path.exists() and not dest_path.exists():
            shutil.copy2(source_path, dest_path)
            print(f"  ✓ Copied: {source} -> {dest}")
        elif dest_path.exists():
            print(f"  ⚠️  Already exists: {dest}")
        else:
            print(f"  ⚠️  Source not found: {source}")

    print()
    print("Migration complete!")


# ============================================================================
# MAIN CLEANUP WORKFLOW
# ============================================================================

def run_cleanup_workflow():
    """
    Complete cleanup workflow with safety checks.
    """

    print()
    print("╔" + "="*68 + "╗")
    print("║" + " "*20 + "PROJECT CLEANUP WIZARD" + " "*27 + "║")
    print("╚" + "="*68 + "╝")
    print()

    # Step 1: Analyze
    print("[STEP 1] Analyzing project...")
    obsolete_files = create_cleanup_plan()
    print()

    input("Press ENTER to continue to Step 2...")
    print()

    # Step 2: Safe archive
    print("[STEP 2] Creating safe archive...")
    archive_dir = safe_archive_cleanup()
    print()

    input("Press ENTER to continue to Step 3...")
    print()

    # Step 3: Organize
    print("[STEP 3] Checking file organization...")
    organize_remaining_files()
    print()

    print("="*70)
    print("CLEANUP SUMMARY")
    print("="*70)
    print()
    print(f"✅ Obsolete files archived to: {archive_dir}/")
    print("✅ New architecture in place")
    print("✅ Old files backed up (not deleted)")
    print()
    print("NEXT STEPS:")
    print()
    print("1. Test dashboard thoroughly:")
    print("   exec(open('test_complete_dashboard.py').read())")
    print()
    print("2. If everything works for a week, delete archive:")
    print(f"   rm -rf {archive_dir}/")
    print()
    print("3. To migrate callbacks/experiment_runner:")
    print("   migrate_remaining_files()")
    print()
    print("="*70)


# Run the workflow
if __name__ == "__main__":
    run_cleanup_workflow()


╔====================================================================╗
║                    PROJECT CLEANUP WIZARD                           ║
╚====================================================================╝

[STEP 1] Analyzing project...
PROJECT CLEANUP ANALYSIS

📋 OBSOLETE FILES (replaced by new architecture):
----------------------------------------------------------------------
  ⚠️  widgets.py                               (NOT FOUND)
  ⚠️  widgets_system_physics_v2.py             (NOT FOUND)
  ⚠️  config_updated.py                        (NOT FOUND)
  ⚠️  PHASE1_NOTEBOOK_INTEGRATION.py           (NOT FOUND)
  ❌ test_phase1.py                           (3.0 KB)
      Reason: Keep or move to tests/
  ❌ test_new_dashboard.py                    (0.6 KB)
      Reason: Keep or move to tests/

✅ FILES TO KEEP (still needed):
----------------------------------------------------------------------
  ✓ config.py                                - Core config (if not migrated to config/

In [18]:
import os

def print_clean_project_tree(startpath):
    # Folders PyCharm hides or that are usually redundant
    exclude_dirs = {'.git', '.idea', '__pycache__', '.venv', 'venv', 'build', 'dist'}
    # File extensions you want to ignore
    exclude_extensions = {'.png', '.pt', '.json', '.pkl'}

    for root, dirs, files in os.walk(startpath):
        # Filter directories in-place to avoid walking into redundant ones
        dirs[:] = [d for d in dirs if d not in exclude_dirs and not d.startswith('.')]

        # Calculate level for indentation
        level = root.replace(startpath, '').count(os.sep)
        indent = '│   ' * level

        # Print current directory name
        folder_name = os.path.basename(root)
        if folder_name == '.':
            folder_name = os.path.basename(os.getcwd())
        print(f'{indent}├── {folder_name}/')

        # Filter files: skip hidden files and specific extensions
        visible_files = [
            f for f in files
            if not f.startswith('.') and not any(f.endswith(ext) for ext in exclude_extensions)
        ]

        # Print files with tree connectors
        sub_indent = '│   ' * (level + 1)
        for i, f in enumerate(visible_files):
            connector = '└── ' if i == len(visible_files) - 1 and not dirs else '├── '
            print(f'{sub_indent}{connector}{f}')

# Run for the current folder
if __name__ == "__main__":
    print_clean_project_tree('.')


├── Session5_clone/
│   ├── config.py
│   ├── data_generation.py
│   ├── LICENSE
│   ├── model_registry.py
│   ├── random_vs_learned_demo.py
│   ├── README.md
│   ├── requirements.txt
│   ├── archive_old_code_20260120_093445/
│   ├── config/
│   │   ├── system_config.py
│   │   └── __init__.py
│   ├── dashboard/
│   │   ├── callbacks.py
│   │   ├── config_manager.py
│   │   ├── experiment_runner.py
│   │   ├── main.py
│   │   ├── plots.py
│   │   ├── README.md
│   │   ├── validators.py
│   │   ├── __init__.py
│   │   ├── components/
│   │   │   ├── buttons.py
│   │   │   ├── results_display.py
│   │   │   ├── stack_manager.py
│   │   │   ├── status_display.py
│   │   │   └── __init__.py
│   │   ├── tabs/
│   │   │   ├── tab_evaluation.py
│   │   │   ├── tab_model.py
│   │   │   ├── tab_physics.py
│   │   │   ├── tab_system.py
│   │   │   ├── tab_training.py
│   │   │   ├── tab_visualization.py
│   │   │   └── __init__.py
│   ├── data/
│   │   ├── data_generation.py
│   │   ├── probe_ge

In [19]:
# ============================================================================
# FINAL SHIM REMOVAL & PROJECT POLISH
# ============================================================================
import os
from pathlib import Path

def remove_shims_and_polish():
    """Remove compatibility shims and clean up empty legacy folders."""

    print("="*70)
    print("FINAL PROJECT POLISH")
    print("="*70)
    print()

    # 1. Backward Compatibility Shims to remove
    shims = [
        'config.py',
        'data_generation.py',
        'model_registry.py'
    ]

    print("[1] Removing Legacy Shims...")
    for shim in shims:
        file_path = Path(shim)
        if file_path.exists():
            # Verify it's a shim (small file) before deleting
            if file_path.stat().st_size < 1000:
                file_path.unlink()
                print(f"  ✓ Deleted: {shim}")
            else:
                print(f"  ⚠️ Skipping: {shim} (File size is large, might contain real code)")
        else:
            print(f"  ○ Not found: {shim}")

    print()

    # 2. Cleanup empty legacy directories
    legacy_dirs = ['experiments']
    print("[2] Cleaning Legacy Directories...")
    for ldir in legacy_dirs:
        dir_path = Path(ldir)
        if dir_path.exists():
            if not any(dir_path.iterdir()):
                dir_path.rmdir()
                print(f"  ✓ Removed empty directory: {ldir}/")
            else:
                print(f"  ⚠️ {ldir}/ is not empty; check contents manually.")
        else:
            print(f"  ○ Not found: {ldir}/")

    print()

    # 3. Final Verification of Structure
    print("[3] Structure Verification:")
    essential_dirs = ['config', 'data', 'models', 'physics', 'dashboard', 'training', 'evaluation', 'tests', 'docs']
    all_good = True
    for d in essential_dirs:
        if Path(d).exists():
            print(f"  ✓ {d:<12} [VERIFIED]")
        else:
            print(f"  ❌ {d:<12} [MISSING]")
            all_good = False

    print("\n" + "="*70)
    if all_good:
        print("🎉 SUCCESS: Your PhD Research Platform is now fully modular and clean!")
    else:
        print("⚠️  WARNING: Some core directories appear to be missing.")
    print("="*70)

if __name__ == "__main__":
    remove_shims_and_polish()

FINAL PROJECT POLISH

[1] Removing Legacy Shims...
  ✓ Deleted: config.py
  ✓ Deleted: data_generation.py
  ✓ Deleted: model_registry.py

[2] Cleaning Legacy Directories...
  ⚠️ experiments/ is not empty; check contents manually.

[3] Structure Verification:
  ✓ config       [VERIFIED]
  ✓ data         [VERIFIED]
  ✓ models       [VERIFIED]
  ✓ physics      [VERIFIED]
  ✓ dashboard    [VERIFIED]
  ✓ training     [VERIFIED]
  ✓ evaluation   [VERIFIED]
  ✓ tests        [VERIFIED]
  ✓ docs         [VERIFIED]

🎉 SUCCESS: Your PhD Research Platform is now fully modular and clean!
